## *Jeu de données IRVE*
-------------------
# Analyse des relations entre champs (intégrité des données)
------------------------------
## Contexte
- clarification du rôle des modèles de données dans les jeux de données (cf mise à jour récente des [guides data.gouv](https://guides.etalab.gouv.fr/qualite/documenter-les-donnees/))
- intégration d'une propriété "relationship" dans les schémas de données ([issue TableSchema](https://github.com/frictionlessdata/specs/issues/803) en cours de validation)
- création d'outils de contrôle des relations entre champs des jeux de données tabulaires (cf usage ci-dessous)

## Objectifs
- valider sur un cas réel l'utilisation d'un modèle de données en complément d'un schéma de données
- identifier les apports que pourraient avoir les contrôles de validation des relations entre champs

## Résultats
- les principaux résultats sont regroupés sur un [tableau de bord](https://nbviewer.org/github/loco-philippe/Environmental-Sensing/blob/main/python/Validation/irve/Analyse/IRVE_indicateurs.ipynb) 
- la formalisation d'un modèle de données facilite la compréhension des données et des relations entre champs
- l'outil de contrôle permet d'améliorer significativement la qualité des données par l'identification d'incohérences de relations
- l'identification des incohérences permet de trouver des stratégies de réduction des écarts (dans l'exemple ci-dessous, on passe 36% d'écart à 3,8 %)  
- l'analyse des données permet de (re)construire le modèle de données qui minimise les incohérences
- les incohérences détectées sur le jeu de données IRVE restent faibles (inférieures à 5 % des point de charge documentés - voir chapitre 4)

## Suite à donner
- Mettre à jour, valider et publier le modèle de données IRVE
- Définir les contrôles supplémentaires à intégrer pour toutes nouvelles données ainsi que pour le jeu complet
- Mettre en oeuvre les outils de contrôle

## Evolutions possibles 
- Ajouter dans les guides d'Etalab un guide pour les modèles de données 
- Intégrer dans les schémas de données la propriété "relationship" en cours de validation,
- Définir un indicateur qui mesure l'écart (existant / attendu) des relations entre champs

## Sommaire
*(liens actifs sur jupyter Notebook ou Nbviewer)*
- [1 - modèle de données](#1---modèle-de-données)
- [2 - Initialisation](#2---Initialisation)
- [3 - Séparation des pdc itinerance et hors itinerance](#3---Séparation-des-pdc-itinerance-et-hors-itinerance)
- [4 - Bilan initial intégrité](#4---Bilan-initial-intégrité)
- [5 - Séparation doublons pdc - date de maj](#5---Séparation-doublons-pdc---date-de-maj)
- [6 - Séparation doublons station - date de maj](#6---Séparation-doublons-station---date-de-maj)
- [7 - Synthèse](#7---Synthèse)
- [8 - Exemples d'erreurs résiduelles](#8---Exemples-d\'erreurs-résiduelles)

Ce Notebook peut être consulté sur [nbviewer](http://nbviewer.org/github/loco-philippe/Environmental-Sensing/tree/main/python/Validation/irve/Analyse)

données utilisées : https://www.data.gouv.fr/fr/datasets/fichier-consolide-des-bornes-de-recharge-pour-vehicules-electriques/    
fichier : "*consolidation-etalab-schema-irve-statique-v-2.2.0-2024xxxx.csv*"

-----------------------
# 1 - modèle de données

Le modèle de données proposé ci-dessous est construit sur la base du schéma de données mis à disposition et du contenu du jeu de données.    
Il est à consolider en fonction de l'expertise des concepteurs et réutilisateurs (voir [guide méthodologique](https://github.com/loco-philippe/Environmental-Sensing/blob/main/property_relationship/FR_methodology.ipynb)).

*Notation:*
- *M : Mandatory - documentation obligatoire*
- *PK : Primary Key - identifiant unique de l'entité*
- *Root : champ fictif associé à une ligne du tableau*

In [1]:
from base64 import b64encode
from IPython.display import Image, display
with open('IRVE_modele.txt', 'r', encoding="utf-8") as f:
    modele = f.read()
display(Image(url="https://mermaid.ink/img/" + b64encode(modele.encode("ascii")).decode("ascii")))

------
# 2 - Initialisation
## initialisation logicielle

In [1]:
from datetime import datetime
import json
from tab_dataset import Cdataset
import pandas as pd
import ntv_pandas as npd
import pathlib

def analyse_integrite(data, schema, affiche=True, indic=True):
    '''analyse les relations du DataFrame 'data' définies dans le schéma 'schema'.
    Le nombre de ligne en erreur par relation (dict) est retourné et optionnellement affiché (paramètre 'affiche=True') . 
    Les lignes en erreur sont optionnellement ajoutées (paramètre 'indic=True') à 'data' sous forme de champs booléens par relation.
    '''
    dic_errors = Cdataset(data).check_relationship(schema)
    dic_count = {name: len(errors) for name, errors in dic_errors.items()}
    if affiche:
        for name, total in dic_count.items():
            print('{:<50} {:>5}'.format(name, total))
    if indic:
        data['ok'] = True
        for name, errors in dic_errors.items():
            data[name] = True
            data.loc[errors, name] = False
            data['ok'] = data['ok'] & data[name] 
    return dic_count

def add_nbre_pdc(data, affiche=True):
    '''ajoute un champ avec le nombre de pdc calculés et le compare au champ 'nbre_pdc'
    Le nombre d'écarts est affiché optionnellement (paramètre 'affiche=True').
    '''
    data['nb_pdc_calc'] = data.groupby('id_station_itinerance')['index'].transform('count')
    nbre_pdc = data['nbre_pdc'].astype('int')
    data['nbre_pdc_ok'] = data['nb_pdc_calc'] == nbre_pdc
    data['calc-nbre'] = data['nb_pdc_calc'] - nbre_pdc
    if affiche:
        ecart_nbre = len(data) - data['nbre_pdc_ok'].sum()
        print('{:<20} {:>5}'.format('nbre_pdc_ko', ecart_nbre))
        print(round(ecart_nbre / len(data) * 100), ' %')

ImportError: Missing optional dependency 'astropy' Use pip or conda to install astropy.

## initialisation des données
- lecture du fichier issu de l'api

In [3]:
dates = ['2023-03-03', '2023-04-17', '2023-05-02', '2023-05-24', '2023-06-17', '2023-07-04', '2023-07-23', '2023-08-08',
         '2023-09-06', '2023-09-19', '2023-10-04', '2023-10-21', '2023-10-31', '2023-11-23', '2023-12-14', '2023-12-29',
         '2024-01-15', '2024-02-07', '2024-02-28', '2024-03-27', '2024-04-24', '2024-05-19', '2024-06-05', '2024-08-02',
         '2024-10-24', '2024-11-27']
date= dates[-1]
log = {'date_irve': date, 
    'file': 'consolidation-etalab-schema-irve-statique-v-2.3.1-'+date[:4]+date[5:7]+date[8:]+'.csv', # après 2024-02-06
    'chemin': str(pathlib.Path(npd.__file__).parent.parent.parent/"Environmental-Sensing"/"python"/"Validation"/"irve"/"Analyse")
      }
irve = pd.read_csv(log['chemin'] + '/' + log['file'], sep=',', low_memory=False, dtype='object')
log['len_irve'] = len(irve)
print('nombre de lignes : ', log['len_irve']) 

nombre de lignes :  115245


## schéma de données
Le schéma de données restreint à la propriété 'relationship' et construit à partir du modèle de données est le suivants :

In [4]:
# complément à inclure dans le schéma de données
schema = {
    'relationships': [
         # relation unicité des pdl
         {"fields": ["id_pdc_itinerance", "index"],                    "link" : "coupled" },   
         # relations inter entités
         {"fields": ["id_station_itinerance", "contact_operateur"],    "link" : "derived" },
         {"fields": ["id_station_itinerance", "nom_enseigne"],         "link" : "derived" },
         {"fields": ["id_station_itinerance", "coordonneesXY"],        "link" : "derived" },
         {"fields": ["id_pdc_itinerance", "id_station_itinerance"],    "link" : "derived" },
         # relations intra entité - station
         {"fields": ["id_station_itinerance", "nom_station"],          "link" : "derived" },
         {"fields": ["id_station_itinerance", "implantation_station"], "link" : "derived" },
         #{"fields": ["id_station_itinerance", "date_maj"],             "link" : "derived" },
         {"fields": ["id_station_itinerance", "nbre_pdc"],             "link" : "derived" },
         {"fields": ["id_station_itinerance", "condition_acces"],      "link" : "derived" },
         {"fields": ["id_station_itinerance", "horaires"],             "link" : "derived" },
         {"fields": ["id_station_itinerance", "station_deux_roues"],   "link" : "derived" },
         # relations intra entité - localisation
         {"fields": ["coordonneesXY", "adresse_station"],              "link" : "derived" }
    ]
}
# liste des champs liés à un controle (relations) et obligatoires (mandatory)
relations = ['index', 'contact_operateur', 'nom_enseigne', 'coordonneesXY', 'adresse_station', 'id_station_itinerance', 
             'nom_station', 'implantation_station', 'nbre_pdc', 'condition_acces', 'horaires', 'station_deux_roues', 
             'id_pdc_itinerance', 'date_maj', 'last_modified']
mandatory = ['contact_operateur', 'nom_enseigne', 'coordonneesXY', 'adresse_station', 'id_station_itinerance', 'nom_station',
             'implantation_station', 'nbre_pdc', 'condition_acces', 'horaires', 'station_deux_roues', 'id_pdc_itinerance', 
             'puissance_nominale', 'prise_type_ef', 'prise_type_2', 'prise_type_combo_ccs', 'prise_type_chademo', 
             'prise_type_autre', 'paiement_acte', 'paiement_autre', 'reservation',  'accessibilite_pmr', 'restriction_gabarit', 
             'date_maj', 'last_modified']
controles = ['index - id_pdc_itinerance', 'contact_operateur - id_station_itinerance', 'nom_enseigne - id_station_itinerance',
             'coordonneesXY - id_station_itinerance', 'id_station_itinerance - id_pdc_itinerance',
             'nom_station - id_station_itinerance', 'implantation_station - id_station_itinerance',
             'nbre_pdc - id_station_itinerance', 'condition_acces - id_station_itinerance', 'horaires - id_station_itinerance',
             'station_deux_roues - id_station_itinerance', 'adresse_station - coordonneesXY']

-----------------------
## 3 - Séparation des pdc itinerance et hors itinerance
- de l'ordre de 3 % des points de charge sont hors itinerance

In [5]:
# séparation des données
data = irve
data[['id_station_itinerance','id_pdc_itinerance','restriction_gabarit']] = data[['id_station_itinerance','id_pdc_itinerance','restriction_gabarit']].astype('string')
#data[['last_modified','date_maj']] = data[['last_modified','date_maj']].astype('datetime64')
data['non_concerne'] = data['id_station_itinerance'].str.contains('oncern') | data['id_pdc_itinerance'].str.contains('oncern')

non_concerne = data[data['non_concerne']].reset_index()['index']
itinerance = data[~data['non_concerne']].reset_index()
itinerance_init = itinerance.loc[:, relations]
log['pdc_hors_itinerance'] = len(non_concerne)
log['pdc_en_itinerance'] = len(itinerance)
print('nombre de pdc hors itinerance : ', log['pdc_hors_itinerance'], 'soit : ', len(non_concerne)/(len(non_concerne)+len(itinerance)))
print('nombre de pdc en itinerance   : ', log['pdc_en_itinerance'])

nombre de pdc hors itinerance :  3497 soit :  0.030344049633389734
nombre de pdc en itinerance   :  111748


In [6]:
# arborescence des champs et nombre de valeurs différentes
print(itinerance_init.npd.analysis().tree())

-1: root-derived (111748)
   0 : index (0 - 111748)
   1 : contact_operateur (111566 - 182)
   2 : nom_enseigne (108050 - 3698)
   3 : coordonneesXY (82727 - 29021)
   4 : adresse_station (83078 - 28670)
   5 : id_station_itineranc (64925 - 46823)
   6 : nom_station (82252 - 29496)
   7 : implantation_station (111743 - 5)
   8 : nbre_pdc (111683 - 65)
   9 : condition_acces (111746 - 2)
   10: horaires (110900 - 848)
   11: station_deux_roues (111742 - 6)
   12: id_pdc_itinerance (10659 - 101089)
   13: date_maj (110977 - 771)
   14: last_modified (111169 - 579)


-----------------------
## 4 - Bilan initial intégrité
- 22 % des lignes présentent un défaut d'intégrité

In [7]:
# séparation données bonnes (itinerance_ok_1) et données résiduelles (itinerance_1)
res = analyse_integrite(itinerance_init, schema)
itinerance_ok_1 = itinerance_init.loc[itinerance_init.ok, relations].reset_index(drop=True)
itinerance_1 = itinerance_init.loc[~itinerance_init.ok, relations].reset_index(drop=True)
itinerance_init = itinerance_init.loc[:, relations]
log['init_ok'] = len(itinerance_ok_1)
log['init_ko'] = len(itinerance_1)
print("\nnombre d'enregistrements sans erreurs : ", log['init_ok'])
print("nombre d'enregistrements avec au moins une erreur : ", log['init_ko'])
print("taux d'erreur : ", round(log['init_ko'] / log['pdc_en_itinerance'] * 100), ' %')

index - id_pdc_itinerance                          21316
contact_operateur - id_station_itinerance           7482
nom_enseigne - id_station_itinerance                8099
coordonneesXY - id_station_itinerance               8767
id_station_itinerance - id_pdc_itinerance          13320
nom_station - id_station_itinerance                 8060
implantation_station - id_station_itinerance        3632
nbre_pdc - id_station_itinerance                    2964
condition_acces - id_station_itinerance               83
horaires - id_station_itinerance                      61
station_deux_roues - id_station_itinerance          7883
adresse_station - coordonneesXY                     2385

nombre d'enregistrements sans erreurs :  86824
nombre d'enregistrements avec au moins une erreur :  24924
taux d'erreur :  22  %


-----------------------
## 5 - Séparation doublons pdc - date de maj
- 35% des pdc en erreur sont liées aux doublons de pdc
- la suppression des doublons permet de réduire de 50% le nombre de lignes erronnées

In [8]:
# séparation doublons pdc (doublons_pdc) et données résiduelles (itinerance_2)
itinerance_1['doublons_pdc'] = itinerance_1.sort_values(by=['date_maj', 'last_modified']).duplicated('id_pdc_itinerance', keep='last')

doublons_pdc = itinerance_1[itinerance_1['doublons_pdc']].loc[:, relations].reset_index(drop=True)['index']
#doublons_pdc = itinerance_1[itinerance_1['doublons_pdc']].loc[:, relations].reset_index(drop=True)
itinerance_2 = itinerance_1[~itinerance_1['doublons_pdc']].loc[:, relations].reset_index(drop=True)
itinerance_1 = itinerance_1.loc[:, relations]
log['doublons_pdc'] = len(doublons_pdc)
log['sans_doublons_pdc'] = len(itinerance_2)
print('nombre de doublons pdc : ', log['doublons_pdc'], ' soit ', round(log['doublons_pdc']/log['init_ko'] * 100), ' %')
print('nombre de pdc sans doublon   : ', log['sans_doublons_pdc'], ' soit ', round(log['sans_doublons_pdc']/log['init_ko'] * 100), ' %')

nombre de doublons pdc :  10659  soit  43  %
nombre de pdc sans doublon   :  14265  soit  57  %


In [9]:
# séparation données bonnes (itinerance_ok_3) et données résiduelles (itinerance_3)
res = analyse_integrite(itinerance_2, schema)
itinerance_ok_3 = itinerance_2.loc[itinerance_2.ok, relations].reset_index(drop=True)
itinerance_3 = itinerance_2.loc[~itinerance_2.ok, relations].reset_index(drop=True)
itinerance_2 = itinerance_2.loc[:, relations]
log['etape3_ok'] = len(itinerance_ok_3)
log['etape3_ko'] = len(itinerance_3)
print("\nnombre d'enregistrements sans erreurs : ", log['etape3_ok'])
print("nombre d'enregistrements avec au moins une erreur : ", log['etape3_ko'])

index - id_pdc_itinerance                              0
contact_operateur - id_station_itinerance             82
nom_enseigne - id_station_itinerance                  89
coordonneesXY - id_station_itinerance                712
id_station_itinerance - id_pdc_itinerance              0
nom_station - id_station_itinerance                  647
implantation_station - id_station_itinerance         191
nbre_pdc - id_station_itinerance                     688
condition_acces - id_station_itinerance               20
horaires - id_station_itinerance                      42
station_deux_roues - id_station_itinerance           506
adresse_station - coordonneesXY                     2123

nombre d'enregistrements sans erreurs :  10551
nombre d'enregistrements avec au moins une erreur :  3714


-----------------------
## 6 - Séparation doublons station - date de maj
- la suppression des anciens pdc permet de réduire de 25% supplémentaire le nombre de lignes erronnées
- les dernières erreurs correspondent à des stations associées à 43 opérateurs et sont liées à des causes multiples

In [10]:
# séparation doublons stations (doublons_stat_maj) et données résiduelles (itinerance_4)
itinerance_3['stat_maj'] = itinerance_3.id_station_itinerance + itinerance_3.date_maj
stat_maj_unique = itinerance_3.sort_values(by='stat_maj').drop_duplicates('id_station_itinerance', keep='last')
itinerance_3['last_stat_maj'] = itinerance_3['stat_maj'].isin(stat_maj_unique['stat_maj'])

doublons_stat_maj = itinerance_3[~itinerance_3['last_stat_maj']].loc[:, relations].reset_index(drop=True)['index']
itinerance_4 = itinerance_3[itinerance_3['last_stat_maj']].loc[:, relations].reset_index(drop=True)
itinerance_3 = itinerance_3.loc[:, relations]
log['doublons_station'] = len(doublons_stat_maj)
log['sans_doublons_station'] = len(itinerance_4)
print('nombre de doublons stations : ', log['doublons_station'], ' soit ', 
      round(log['doublons_station']/log['init_ko'] * 100), ' %')
print('nombre de pdc sans doublon   : ', log['sans_doublons_station'])

nombre de doublons stations :  198  soit  1  %
nombre de pdc sans doublon   :  3516


In [11]:
# séparation données bonnes (itinerance_ok_5) et données résiduelles (itinerance_5 / itinerance_5_full)
res = analyse_integrite(itinerance_4, schema)
itinerance_ok_5 = itinerance_4.loc[itinerance_4.ok, relations].reset_index(drop=True)
itinerance_5_full = itinerance_4.loc[~itinerance_4.ok].reset_index(drop=True)
itinerance_5 = itinerance_5_full.loc[:, relations]
#itinerance_4 = itinerance_4.loc[:, relations]
log['etape5_ok'] = len(itinerance_ok_5)
log['etape5_ko'] = len(itinerance_5)
print("\nnombre d'enregistrements sans erreurs : ", log['etape5_ok'])
print("nombre d'enregistrements avec au moins une erreur : ", log['etape5_ko'], ' soit ', 
      round(log['etape5_ko']/log['etape3_ko'] * 100), ' %')

index - id_pdc_itinerance                              0
contact_operateur - id_station_itinerance              2
nom_enseigne - id_station_itinerance                   9
coordonneesXY - id_station_itinerance                439
id_station_itinerance - id_pdc_itinerance              0
nom_station - id_station_itinerance                  473
implantation_station - id_station_itinerance         159
nbre_pdc - id_station_itinerance                     419
condition_acces - id_station_itinerance                0
horaires - id_station_itinerance                      39
station_deux_roues - id_station_itinerance           448
adresse_station - coordonneesXY                     2035

nombre d'enregistrements sans erreurs :  151
nombre d'enregistrements avec au moins une erreur :  3365  soit  91  %


In [12]:
# structure des données présentant une erreur et nombre de valeurs
print(itinerance_5.npd.analysis().tree())

-1: root-derived (3365)
   0 : index (0 - 3365)
   1 : contact_operateur (3308 - 57)
   2 : nom_enseigne (3168 - 197)
      9 : condition_acces (195 - 2)
   3 : coordonneesXY (2730 - 635)
   4 : adresse_station (2351 - 1014)
   5 : id_station_itineranc (1930 - 1435)
      13: date_maj (1317 - 118)
   6 : nom_station (2430 - 935)
   7 : implantation_station (3360 - 5)
   8 : nbre_pdc (3332 - 33)
   10: horaires (3298 - 67)
   11: station_deux_roues (3360 - 5)
   12: id_pdc_itinerance (0 - 3365)
   14: last_modified (3267 - 98)


In [13]:
# nb maxi d'erreurs
itinerance_4['somme'] = 0
for name in res.keys():
    itinerance_4['somme'] += 1 - itinerance_4[name]
erreurs = max(itinerance_4['somme'])
maxi = itinerance_4[itinerance_4.somme >= erreurs]
print("nombre d'enregistrements avec ", erreurs, " erreurs : ", len(maxi))

nombre d'enregistrements avec  4  erreurs :  4


In [14]:
# principaux opérateurs contributeur de défauts
operateurs = list(itinerance_5.drop_duplicates('contact_operateur')['contact_operateur'])
erreurs_op = [len(itinerance_5.loc[itinerance_5.contact_operateur == operateur]) for operateur in operateurs]
err_op_5, oper_5 = tuple(zip(*(sorted(zip(erreurs_op, operateurs), reverse=True))[:5]))

print(len(err_op_5), ' opérateurs représentent : ', round(sum(err_op_5) / sum(erreurs_op) * 100), ' % des écarts\n')
for err, op in zip(err_op_5, oper_5):
    print(err, op)

5  opérateurs représentent :  65  % des écarts

731 support@alizecharge.fr
627 supervision-ev.france@totalenergies.com
317 sav@izivia.com
296 a.buffler@bouygues-es.com
218 info@ionity.eu


-----------------------
## 7 - Synthèse


### fichiers
Génération des fichiers intégrant les défauts d'intégrité :
- fichier csv des lignes résiduelles à traiter (IRVE_itinerance_residuel)
- fichier csv des données itinerance avec indicateur des données à corriger ou à ignorer (IRVE_itinerance_complet)
- fichier csv des données itinerance valides (IRVE_itinerance_valide)
- fichier csv des doublons (IRVE_itinerance_doublons)

In [15]:
# consolidation des données
itinerance['doublons_stat_maj'] = itinerance['index'].isin(doublons_stat_maj)
itinerance['doublons_pdc'] = itinerance['index'].isin(doublons_pdc)
itinerance['lignes_a_corriger'] = itinerance['index'].isin(itinerance_5['index'])
itinerance['doublons_a_supprimer'] = itinerance['doublons_stat_maj'] | itinerance['doublons_pdc']
itinerance['lignes_ko'] = itinerance['doublons_a_supprimer'] | itinerance['lignes_a_corriger']
print('total des lignes à corriger : ', itinerance['lignes_a_corriger'].sum())
itinerance_doublons = itinerance[itinerance['doublons_a_supprimer']].reset_index(drop=True)
print('total des doublons à supprimer : ', len(itinerance_doublons))
itinerance_ok = itinerance[~itinerance['lignes_ko']].reset_index(drop=True)
print('nombre de pdc avec controles ok : ', len(itinerance_ok))

total des lignes à corriger :  3365
total des doublons à supprimer :  10857
nombre de pdc avec controles ok :  97526


In [16]:
# génération des fichiers
extension = log['date_irve'] +'.csv'
itinerance_5_full.to_csv('IRVE_itinerance_residuel' + extension)
itinerance.to_csv('IRVE_itinerance_complet' + extension)
itinerance_ok.to_csv('IRVE_itinerance_valide' + extension)
itinerance_doublons.to_csv('IRVE_itinerance_doublons' + extension)
log['IRVE_itinerance_residuel' + extension] = len(itinerance_5_full)
log['IRVE_itinerance_complet' + extension] = len(itinerance)
log['IRVE_itinerance_valide' + extension] = len(itinerance_ok)
log['IRVE_itinerance_valide_stat' + extension] = len(itinerance_ok.drop_duplicates('id_station_itinerance', keep='last'))
log['IRVE_itinerance_doublons' + extension] = len(itinerance_doublons)

In [17]:
l_prefix = list(pd.read_csv(log['chemin'] + '/' + 'Operations_Recharge_16-07-2024.csv', sep=';', low_memory=False, dtype='object')['Identifiant'])
l_prefix[0]

'FR073'

In [18]:
import numpy as np

df = pd.DataFrame({'a':["er", "rt", "", "ert"], 'b': [10, np.nan, 20, 1]})

df['b'] = df['b'].replace(np.nan, 0)
df['a'] = df['a'].replace("", "empty")
df
itinerance_ok['cable_t2_attache'].astype('bool')


0        True
1        True
2        True
3        True
4        True
         ... 
97521    True
97522    True
97523    True
97524    True
97525    True
Name: cable_t2_attache, Length: 97526, dtype: bool

In [19]:
from pydantic import (BaseModel, EmailStr, PastDate, PositiveFloat, PositiveInt, ValidationError)
from pydantic_extra_types.phone_numbers import PhoneNumber
from copy import copy
import re

PREFIX = list(pd.read_csv(log['chemin'] + '/' + 'Operations_Recharge_16-07-2024.csv', sep=';', low_memory=False, dtype='object')['Identifiant'])
    
NUMPDL = re.compile(r"^\d{14}$")

class FrenchPhoneNumber(PhoneNumber):
    """A phone number with french defaults."""
    default_region_code = "FR"
    
class EmailStr(BaseModel):
    x: EmailStr

class PastDate(BaseModel):
    x: PastDate

class PositiveInt(BaseModel):
    x: PositiveInt

class FrenchPhoneNumber(BaseModel):
    x: FrenchPhoneNumber

class NumPdl:
    def __init__(self, x=None):
        self.x = x
        if NUMPDL.fullmatch(x) is None:
            raise ValidationError

def check_value(value, clas=None, default=None):    
    try:
        clas(x=value)
    except:
        return default
    return value

def set_prefix(pref):    
    return pref if pref in PREFIX else PREFIX[0]
    
def pandas_to_qualicharge(data):
    '''
    convertit les champs suivants (et applique une valeur par défaut si erreur) :
        - contact_amenageur: EmailStr ('amenageur@email.fr')
        - contact_operateur: EmailStr ('example@email.fr')
        - telephone_operateur: FrenchPhoneNumber (06-06-06-06-06)
        - nbre_pdc: PositiveInt (0)
        - date_maj: PastDate ('1900-01-01')
        - num_pdl: "^\d{14}$" ('00001234567890')
        - date_mise_en_service: PastDate ('1900-01-01')
        - puissance_nominale: PositiveFloat (0.0)
    et applique le prefixe des stations aux pdc.
    '''
    quali = copy(data)

    quali['paiement_autre'] = quali['paiement_autre'].astype('bool')
    quali['cable_t2_attache'] = quali['cable_t2_attache'].astype('bool')
    quali['raccordement'] = quali['raccordement'].replace(np.nan, 'Direct')
    quali['siren_amenageur'] = quali['siren_amenageur'].astype('string').replace(["", np.nan], "000000000")
    quali['code_insee_commune'] = quali['code_insee_commune'].astype('string').replace(["", np.nan], "99999")
    
    quali['contact_amenageur'] = quali['contact_amenageur'].apply(check_value,  clas=EmailStr, default='amenageur@email.fr')
    quali['contact_operateur'] = quali['contact_operateur'].apply(check_value,  clas=EmailStr, default='operateur@email.fr')
    quali['telephone_operateur'] = quali['telephone_operateur'].apply(check_value,  clas=FrenchPhoneNumber, default='06-06-06-06-06')
    quali['nbre_pdc'] = quali['nbre_pdc'].apply(check_value,  clas=PositiveInt, default=0)
    quali['date_maj'] = quali['date_maj'].apply(check_value,  clas=PastDate, default='1900-01-01')
    quali['num_pdl'] = quali['num_pdl'].apply(check_value,  clas=NumPdl, default='00001234567890')
    quali['date_mise_en_service'] = quali['date_mise_en_service'].apply(check_value,  clas=PastDate, default='1900-01-01')
    quali['puissance_nominale'] = quali['puissance_nominale'].apply(check_value,  clas=PositiveFloat, default=0.01)
    prefix = quali['id_station_itinerance'].astype('str').str.slice(stop=5).apply(set_prefix)
    quali['id_station_itinerance'] = prefix.str.cat(quali['id_station_itinerance'].astype('str').str.slice(start=5))
    quali['id_pdc_itinerance'] = prefix.str.cat(quali['id_pdc_itinerance'].astype('str').str.slice(start=5))

    quali['restriction_gabarit'] = quali['restriction_gabarit'].astype('string')
    return quali

itinerance_quali = pandas_to_qualicharge(itinerance_ok)

In [20]:
type(itinerance_quali.loc[85, 'restriction_gabarit'])
itinerance_quali['paiement_autre'].unique()

array([ True])

In [22]:
itinerance_quali.loc[:80, relations]


,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
0,0,roaming@zunder.com,Zunder,"[-0.056488 , 48.723084]","D924, Aire du Pays d’Argentan, Écouché-les-V...",FR073P2067778232665043277,Zunder/74654,Station dédiée à la recharge rapide,12,Accès libre,24/7,False,FR073E1111ER1,2024-09-05,2024-10-15T03:01:24+00:00
1,1,roaming@zunder.com,Zunder,"[-0.056488 , 48.723084]","D924, Aire du Pays d’Argentan, Écouché-les-V...",FR073P2067778232665043277,Zunder/74654,Station dédiée à la recharge rapide,12,Accès libre,24/7,False,FR073E1111ER10,2024-09-05,2024-10-15T03:01:24+00:00
2,2,roaming@zunder.com,Zunder,"[-0.056488 , 48.723084]","D924, Aire du Pays d’Argentan, Écouché-les-V...",FR073P2067778232665043277,Zunder/74654,Station dédiée à la recharge rapide,12,Accès libre,24/7,False,FR073E1111ER11,2024-09-05,2024-10-15T03:01:24+00:00
3,3,roaming@zunder.com,Zunder,"[-0.056488 , 48.723084]","D924, Aire du Pays d’Argentan, Écouché-les-V...",FR073P2067778232665043277,Zunder/74654,Station dédiée à la recharge rapide,12,Accès libre,24/7,False,FR073E1111ER12,2024-09-05,2024-10-15T03:01:24+00:00
4,4,roaming@zunder.com,Zunder,"[-0.056488 , 48.723084]","D924, Aire du Pays d’Argentan, Écouché-les-V...",FR073P2067778232665043277,Zunder/74654,Station dédiée à la recharge rapide,12,Accès libre,24/7,False,FR073E1111ER2,2024-09-05,2024-10-15T03:01:24+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,76,roaming@zunder.com,Zunder,"[0.891121 , 47.301879]","A85, km115, Athée-sur-Cher 37270 France",FR073P6119664770224438902,Zunder/122289,Station dédiée à la recharge rapide,39,Accès libre,24/7,False,FR073EFR3701ER31,2024-09-05,2024-10-15T03:01:24+00:00
77,77,roaming@zunder.com,Zunder,"[0.891121 , 47.301879]","A85, km115, Athée-sur-Cher 37270 France",FR073P6119664770224438902,Zunder/122289,Station dédiée à la recharge rapide,39,Accès libre,24/7,False,FR073EFR3701ER32,2024-09-05,2024-10-15T03:01:24+00:00
78,78,roaming@zunder.com,Zunder,"[0.891121 , 47.301879]","A85, km115, Athée-sur-Cher 37270 France",FR073P6119664770224438902,Zunder/122289,Station dédiée à la recharge rapide,39,Accès libre,24/7,False,FR073EFR3701ER33,2024-09-05,2024-10-15T03:01:24+00:00
79,79,roaming@zunder.com,Zunder,"[0.891121 , 47.301879]","A85, km115, Athée-sur-Cher 37270 France",FR073P6119664770224438902,Zunder/122289,Station dédiée à la recharge rapide,39,Accès libre,24/7,False,FR073EFR3701ER4,2024-09-05,2024-10-15T03:01:24+00:00


In [23]:
itinerance_quali.loc[:, relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
0,0,roaming@zunder.com,Zunder,"[-0.056488 , 48.723084]","D924, Aire du Pays d’Argentan, Écouché-les-V...",FR073P2067778232665043277,Zunder/74654,Station dédiée à la recharge rapide,12,Accès libre,24/7,False,FR073E1111ER1,2024-09-05,2024-10-15T03:01:24+00:00
1,1,roaming@zunder.com,Zunder,"[-0.056488 , 48.723084]","D924, Aire du Pays d’Argentan, Écouché-les-V...",FR073P2067778232665043277,Zunder/74654,Station dédiée à la recharge rapide,12,Accès libre,24/7,False,FR073E1111ER10,2024-09-05,2024-10-15T03:01:24+00:00
2,2,roaming@zunder.com,Zunder,"[-0.056488 , 48.723084]","D924, Aire du Pays d’Argentan, Écouché-les-V...",FR073P2067778232665043277,Zunder/74654,Station dédiée à la recharge rapide,12,Accès libre,24/7,False,FR073E1111ER11,2024-09-05,2024-10-15T03:01:24+00:00
3,3,roaming@zunder.com,Zunder,"[-0.056488 , 48.723084]","D924, Aire du Pays d’Argentan, Écouché-les-V...",FR073P2067778232665043277,Zunder/74654,Station dédiée à la recharge rapide,12,Accès libre,24/7,False,FR073E1111ER12,2024-09-05,2024-10-15T03:01:24+00:00
4,4,roaming@zunder.com,Zunder,"[-0.056488 , 48.723084]","D924, Aire du Pays d’Argentan, Écouché-les-V...",FR073P2067778232665043277,Zunder/74654,Station dédiée à la recharge rapide,12,Accès libre,24/7,False,FR073E1111ER2,2024-09-05,2024-10-15T03:01:24+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97521,115126,advenir@zeborne.com,TOYOTA NICE - NOVELLIPSE,"[7.201547, 43.6885902]",108 Avenue Simone Veil 06200 Nice,FRZTLE24DC55361,Toyota - Nice - 24kW DC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:30,Mo-Fr 13:30–19:00,Sat 09:00-...",false,FRZTLE24DC55361,2021-02-24,2024-01-19T07:46:20.729000+00:00
97522,115127,advenir@zeborne.com,GARAGE NELLO CHELI - BEAUNE,"[4.86082418, 47.0236522]",RUE LUCIEN PERRIAUX ZI DE VIGNOLES 21200 Beaune,FRZTLE24DC61318,Toyota - Beaune - 24kW DC,Parking public,1,Accès libre,"Mo-Fr 07:45-12:00,Mo-Fr 13:45-19:00,Sat 09:00-...",false,FRZTLE24DC61318,2021-05-24,2024-01-19T07:46:19.511000+00:00
97523,115128,advenir@zeborne.com,GARAGE NELLO CHELI - BEAUNE,"[4.86082418, 47.0236522]",RUE LUCIEN PERRIAUX ZI DE VIGNOLES 21200 Beaune,FRZTLE24DC61319,Toyota - Beaune - 24kW DC,Parking public,1,Accès libre,"Mo-Fr 07:45-12:00,Mo-Fr 13:45-19:00,Sat 09:00-...",false,FRZTLE24DC61319,2021-05-24,2024-01-19T07:46:19.511000+00:00
97524,115129,advenir@zeborne.com,TOYOTA ANNEMASSE - DEGENEVE,"[6.2174783, 46.1858677]",8 RUE DE L'INDUSTRIE 74240 Gaillard,FRZTLE24DC63446,Toyota - Annemasse - 24kW DC,Parking public,1,Accès libre,"Mo-Fr 08:00-12:00,Mo-Fr 14:00-19:00,Sat 09:00-...",false,FRZTLE24DC63446,2021-06-16,2024-01-19T07:46:18.987000+00:00


In [24]:
import csv
itinerance_quali.to_csv('IRVE_itinerance_valide_quali' + extension, index=False, quoting=csv.QUOTE_ALL, na_rep='empty')

In [25]:
itinerance_quali['paiement_autre'].to_csv('test.csv', index=False, quoting=csv.QUOTE_ALL, na_rep='empty')

In [26]:
for op in oper_5:
    print('IRVE_itinerance_residuel' + '_' + op + '_' + extension)
    itinerance_5_full.loc[itinerance_5_full.contact_operateur == op].to_csv('IRVE_itinerance_residuel' + '_' + op + '_' + extension)

IRVE_itinerance_residuel_support@alizecharge.fr_2024-10-24.csv
IRVE_itinerance_residuel_supervision-ev.france@totalenergies.com_2024-10-24.csv
IRVE_itinerance_residuel_sav@izivia.com_2024-10-24.csv
IRVE_itinerance_residuel_a.buffler@bouygues-es.com_2024-10-24.csv
IRVE_itinerance_residuel_info@ionity.eu_2024-10-24.csv


### vérification de l'intégrité

In [27]:
# vérification de l'absence d'erreurs
res = analyse_integrite(itinerance_ok.loc[:, relations], schema, affiche=False)
log['bilan_erreurs'] = sum(res.values())
log['date'] = datetime.now().isoformat()
print('bilan intégrité :')
print('    erreurs : ', log['bilan_erreurs'])

bilan intégrité :
    erreurs :  0


In [28]:
# structure des données bonnes
#print(Cdataset(itinerance_ok.loc[:, mandatory]).tree())
print(itinerance_ok.loc[:, mandatory].npd.analysis().tree())

-1: root-derived (97526)
   4 : id_station_itineranc (55268 - 42258)
      0 : contact_operateur (42094 - 164)
      1 : nom_enseigne (38751 - 3507)
      2 : coordonneesXY (16732 - 25526)
         3 : adresse_station (605 - 24921)
      5 : nom_station (16708 - 25550)
      6 : implantation_station (42253 - 5)
      7 : nbre_pdc (42193 - 65)
      8 : condition_acces (42256 - 2)
      9 : horaires (41491 - 767)
      10: station_deux_roues (42252 - 6)
      18: paiement_acte (42250 - 8)
      22: restriction_gabarit (42155 - 103)
   11: id_pdc_itinerance (0 - 97526)
   12: puissance_nominale (97378 - 148)
   13: prise_type_ef (97518 - 8)
   14: prise_type_2 (97519 - 7)
   15: prise_type_combo_ccs (97519 - 7)
   16: prise_type_chademo (97519 - 7)
   17: prise_type_autre (97519 - 7)
   19: paiement_autre (97517 - 9)
   20: reservation (97519 - 7)
   21: accessibilite_pmr (97522 - 4)
   23: date_maj (96908 - 618)
   24: last_modified (97027 - 499)


### Indicateurs

In [29]:
# génération des indicateurs liés aux règles
regles = ['Pdc non unique', 'Station multi-operateurs', 'Station multi-enseignes', 'Station multi-localisations', 
          'Pdc multi-stations', 'station avec plusieurs noms', 'station multi-implantations', 
          'nombre de pdc par station incoherent', 'station multi-acces', 'station multi-horaires', 
          'acces deux-roues incoherent', 'localisation multi-adresses']
principal = [16, 17, 18, 19, 20]
secondaire = [21, 22, 23, 24, 25, 26, 27]
irve = itinerance_5_full # residuel
total = len(irve)
indic = {}

irve['principal'] = True
for ind in principal:
    irve['principal'] &= irve.iloc[:,ind]
    indic[regles[ind-16]] = int(total - irve.iloc[:,ind].sum())
irve['secondaire'] = True
for ind in secondaire:
    irve['secondaire'] &= irve.iloc[:,ind]
    indic[regles[ind-16]] = int(total - irve.iloc[:,ind].sum())
irve['secondaire'] |= (~irve['principal'] & ~irve['secondaire'])
irve['verif'] = irve['principal'] & irve['secondaire']
indic['principal pdc'] = int(total - irve['principal'].sum())
indic['secondaire pdc'] = int(total - irve['secondaire'].sum())

irve_p = irve[~irve['principal']].drop_duplicates('id_station_itinerance').reset_index(drop=True)
irve_s = irve[~irve['secondaire']].drop_duplicates('id_station_itinerance').reset_index(drop=True)
indic['principal stat'] = len(irve_p)
indic['secondaire stat'] = len(irve_s)

In [30]:
# ajout des principaux opérateurs contributeur de défauts
indic['operateurs'] = list(oper_5)
indic['erreurs_operateurs'] = list(err_op_5)

In [31]:
# ajout de l'indicateur d'écart entre nombre de pdc calculés et saisis
add_nbre_pdc(irve, affiche=True)
indic['nb_pdc_calc-nbre_pdc'] = int(irve['calc-nbre'].sum())

nbre_pdc_ko           1310
39  %


In [32]:
# indicateur d'écart entre nombre de pdc calculés et saisis sur les données sans doublon
itinerance_sans_doublons = itinerance[~itinerance['doublons_a_supprimer']].reset_index(drop=True)
add_nbre_pdc(itinerance_sans_doublons, affiche=True)

nbre_pdc_ko          34679
34  %


In [33]:
len(itinerance_4)

3516

In [34]:
# stockage des indicateurs
log |= indic
log_file = log['chemin'] + '/' + 'logfile.txt'
#with open('logfile.txt', 'a', encoding="utf-8") as f:
with open(log_file, 'a', encoding="utf-8") as f:
    f.write(json.dumps(log) + '\n')

In [35]:
itinerance_5_full.columns[16:28]

Index(['index - id_pdc_itinerance',
       'contact_operateur - id_station_itinerance',
       'nom_enseigne - id_station_itinerance',
       'coordonneesXY - id_station_itinerance',
       'id_station_itinerance - id_pdc_itinerance',
       'nom_station - id_station_itinerance',
       'implantation_station - id_station_itinerance',
       'nbre_pdc - id_station_itinerance',
       'condition_acces - id_station_itinerance',
       'horaires - id_station_itinerance',
       'station_deux_roues - id_station_itinerance',
       'adresse_station - coordonneesXY'],
      dtype='object')

-----------------------
## 8 - Exemples d'erreurs résiduelles 

### Erreurs multiples 
- exemple avec le nombre maximal d'erreurs (3) (109 pdc)
- exemple station 'FR55CP92140' avec 29 pdc ! : 5 coordonnées XY, 5 nom_station, 2 nbre_pdc)

In [36]:
maxi.loc[:, relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
1102,59400,support@mobilize-ps.fr,JRA SAUMUR,"[-0.059981, 47.274681]","\tBd Vallerots, 49400 Saumur",FRMBZEEVHJW,JRA SAUMUR,Parking privé à usage public,1,Accès libre,"Mo-Fr 07:30-12:00,Mo-Fr 14:00-19:00",False,FRMBZEEVHJW1,2024-07-10,2024-10-23T15:12:49.416000+00:00
1103,59401,support@mobilize-ps.fr,JRA Saumur,"[-0.05996536452405448, 47.27462246981463]","Bd Vallerots, 49400 Saumur",FRMBZEEVHJW,JRA Saumur,Parking privé à usage public,1,Accès libre,24/7,False,FRMBZEEVHJW2,2024-07-10,2024-10-23T15:12:49.416000+00:00
1113,59438,support@mobilize-ps.fr,BODEMER BAYEUX,"[-0.700226002831211, 49.27084281571326]","16 Bd Sadi Carnot, 14400 Bayeux",FRMBZESMYQC,BODEMER Bayeux,Parking privé à usage public,1,Accès libre,24/7,False,FRMBZESMYCQ1,2024-06-03,2024-10-23T15:12:49.416000+00:00
1114,59439,support@mobilize-ps.fr,BODEMER - SCAUTO BAYEUX,"[-0.700259, 49.270901]","\t16 Bd Sadi Carnot, 14400 Bayeux",FRMBZESMYQC,\t\tScauto Bayeux,Parking privé à usage public,1,Accès libre,"Mo-Fr 07:30-12:00,Mo-Fr 14:00-19:00",False,FRMBZESMYQC1,2024-06-03,2024-10-23T15:12:49.416000+00:00


In [37]:
itinerance_4.loc[itinerance_4.id_station_itinerance == 'FR55CP92140', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified


### Repartition des defauts par operateur
Répartition pour les 5 opérateurs présentant le plus de défauts

In [38]:
print('PdC avec une incohérence : ')
for operateur in oper_5:
    print('\n', sum(itinerance_5.contact_operateur == operateur))
    print('Répartition des incohérences ', operateur, ' :')
    for cont in controles:
        #print(cont, sum((itinerance_4['contact_operateur'] == 'supervision-ev.france@totalenergies.com') & ~(itinerance_4[cont])))
        print('{:<50} {:>5}'.format(cont, sum((itinerance_4['contact_operateur'] == operateur) & ~(itinerance_4[cont]))))

PdC avec une incohérence : 

 731
Répartition des incohérences  support@alizecharge.fr  :
index - id_pdc_itinerance                              0
contact_operateur - id_station_itinerance              0
nom_enseigne - id_station_itinerance                   0
coordonneesXY - id_station_itinerance                217
id_station_itinerance - id_pdc_itinerance              0
nom_station - id_station_itinerance                  312
implantation_station - id_station_itinerance          44
nbre_pdc - id_station_itinerance                     176
condition_acces - id_station_itinerance                0
horaires - id_station_itinerance                       0
station_deux_roues - id_station_itinerance             0
adresse_station - coordonneesXY                      425

 627
Répartition des incohérences  supervision-ev.france@totalenergies.com  :
index - id_pdc_itinerance                              0
contact_operateur - id_station_itinerance              0
nom_enseigne - id_station_itinera

### Defauts operateur ALIZECHARGE
Les défauts identifiés concernent des stations utilisant un même identifiant. Par exemple, la station 'FRADPP91479018' contient 82 points de recharge avec plusieurs adresses et un nombre de points de rechage variable. Cet exemple concerne des stations implantées sur plusieurs étages d'un parking (on devrait donc a minima identifier une station par étage).

In [42]:
itinerance_4.loc[itinerance_4['contact_operateur'] == 'support@alizecharge.fr', relations][50:200]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
97,2203,support@alizecharge.fr,ADP Groupe,"[2.362554,48.728787]","2 Rue des Transporteurs, 94390 Paray-Vieille-P...",FRADPE914790183,ORY - Parking P3 SILO - Niveau 4,Parking public,16,Accès libre,Mo-Su 00:00-23:57,False,FRADPE9147901839,2024-10-17,2024-10-17T15:55:47.131000+00:00
98,2204,support@alizecharge.fr,ADP Groupe,"[2.362554,48.728787]","2 Rue des Transporteurs, 94390 Paray-Vieille-P...",FRADPE91479018,ORY - Parking P3 SILO - Niveau 2,Parking public,16,Accès libre,Mo-Su 00:00-23:57,False,FRADPE914790184,2024-10-17,2024-10-17T15:55:47.131000+00:00
99,2205,support@alizecharge.fr,ADP Groupe,"[2.362554,48.728787]","2 Rue des Transporteurs, 94390 Paray-Vieille-P...",FRADPE914790184,ORY - Parking P3 SILO - Niveau 4,Parking public,16,Accès libre,Mo-Su 00:00-23:57,False,FRADPE9147901840,2024-10-17,2024-10-17T15:55:47.131000+00:00
100,2206,support@alizecharge.fr,ADP Groupe,"[2.362554,48.728787]","2 Rue des Transporteurs, 94390 Paray-Vieille-P...",FRADPE914790184,ORY - Parking P3 SILO - Niveau 4,Parking public,16,Accès libre,Mo-Su 00:00-23:57,False,FRADPE9147901841,2024-10-17,2024-10-17T15:55:47.131000+00:00
101,2207,support@alizecharge.fr,ADP Groupe,"[2.362554,48.728787]","2 Rue des Transporteurs, 94390 Paray-Vieille-P...",FRADPE914790184,ORY - Parking P3 SILO - Niveau 4,Parking public,16,Accès libre,Mo-Su 00:00-23:57,False,FRADPE9147901842,2024-10-17,2024-10-17T15:55:47.131000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1087,56797,support@alizecharge.fr,Grenoble-Alpes Métropole,"[5.750434,45.19941]","Avenue De L'Obiou, 38700 LA TRONCHE",FRM38E38516001,LA TRONCHE - Avenue De L'Obiou,Voirie,2,Accès libre,Mo-Su 00:00-23:57,False,FRM38E385160014,2024-10-17,2024-10-17T15:55:47.131000+00:00
1380,66082,support@alizecharge.fr,EFFIA France,"[5.3810096,43.3023197]","1 AVENUE PIERRE SEMARD, 13002 MARSEILLE",FRP01E13201001,Parking gare de Marseille Saint-Charles P1 - E...,Parking public,4,Accès libre,Mo-Su 00:00-23:57,False,FRP01E1320100121,2024-10-17,2024-10-17T15:55:47.131000+00:00
1381,66083,support@alizecharge.fr,EFFIA France,"[5.3810096,43.3023197]","1 AVENUE PIERRE SEMARD, 13002 MARSEILLE",FRP01E13201001,Parking gare de Marseille Saint-Charles P1 - E...,Parking public,4,Accès libre,Mo-Su 00:00-23:57,False,FRP01E1320100122,2024-10-17,2024-10-17T15:55:47.131000+00:00
1382,66084,support@alizecharge.fr,EFFIA France,"[5.3810096,43.3023197]","1 AVENUE PIERRE SEMARD, 13002 MARSEILLE",FRP01E13201001,Parking gare de Marseille Saint-Charles P1 - E...,Parking public,4,Accès libre,Mo-Su 00:00-23:57,False,FRP01E1320100123,2024-10-17,2024-10-17T15:55:47.131000+00:00


In [88]:
# example [2.362554,48.728787]
adp = itinerance_4.loc[itinerance_4.coordonneesXY == '[2.362554,48.728787]']
adresses = list(adp.drop_duplicates('adresse_station')['adresse_station'])
print(adresses)

print(adp.loc[~adp['nom_station - id_station_itinerance'], ['nom_station', 'id_station_itinerance']])

print(adp.loc[~adp['implantation_station - id_station_itinerance'], ['implantation_station', 'id_station_itinerance']])

print(adp.loc[~adp['nbre_pdc - id_station_itinerance'], ['nbre_pdc', 'id_station_itinerance']])

itinerance_4.loc[itinerance_4.coordonneesXY == '[2.362554,48.728787]', relations]

['2 Rue des Transporteurs, 94390 Paray-Vieille-Poste', '2 Rue des Transporteurs, 94390  Paray-Vieille-Poste']
                          nom_station id_station_itinerance
66   ORY - Parking P3 SILO - Niveau 2       FRADPE914790181
67   ORY - Parking P3 SILO - Niveau 2       FRADPE914790181
68   ORY - Parking P3 SILO - Niveau 2       FRADPE914790181
69   ORY - Parking P3 SILO - Niveau 2       FRADPE914790181
70   ORY - Parking P3 SILO - Niveau 2       FRADPE914790181
71   ORY - Parking P3 SILO - Niveau 2       FRADPE914790181
72   ORY - Parking P3 SILO - Niveau 2       FRADPE914790181
73   ORY - Parking P3 SILO - Niveau 3       FRADPE914790181
74   ORY - Parking P3 SILO - Niveau 3       FRADPE914790181
75   ORY - Parking P3 SILO - Niveau 3       FRADPE914790181
88   ORY - Parking P3 SILO - Niveau 3       FRADPE914790183
89   ORY - Parking P3 SILO - Niveau 3       FRADPE914790183
90   ORY - Parking P3 SILO - Niveau 3       FRADPE914790183
91   ORY - Parking P3 SILO - Niveau 4       FRADPE

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
65,2171,support@alizecharge.fr,ADP Groupe,"[2.362554,48.728787]","2 Rue des Transporteurs, 94390 Paray-Vieille-P...",FRADPE91479018,ORY - Parking P3 SILO - Niveau 2,Parking public,16,Accès libre,Mo-Su 00:00-23:57,False,FRADPE914790181,2024-10-17,2024-10-17T15:55:47.131000+00:00
66,2172,support@alizecharge.fr,ADP Groupe,"[2.362554,48.728787]","2 Rue des Transporteurs, 94390 Paray-Vieille-P...",FRADPE914790181,ORY - Parking P3 SILO - Niveau 2,Parking public,16,Accès libre,Mo-Su 00:00-23:57,False,FRADPE9147901810,2024-10-17,2024-10-17T15:55:47.131000+00:00
67,2173,support@alizecharge.fr,ADP Groupe,"[2.362554,48.728787]","2 Rue des Transporteurs, 94390 Paray-Vieille-P...",FRADPE914790181,ORY - Parking P3 SILO - Niveau 2,Parking public,16,Accès libre,Mo-Su 00:00-23:57,False,FRADPE9147901811,2024-10-17,2024-10-17T15:55:47.131000+00:00
68,2174,support@alizecharge.fr,ADP Groupe,"[2.362554,48.728787]","2 Rue des Transporteurs, 94390 Paray-Vieille-P...",FRADPE914790181,ORY - Parking P3 SILO - Niveau 2,Parking public,16,Accès libre,Mo-Su 00:00-23:57,False,FRADPE9147901812,2024-10-17,2024-10-17T15:55:47.131000+00:00
69,2175,support@alizecharge.fr,ADP Groupe,"[2.362554,48.728787]","2 Rue des Transporteurs, 94390 Paray-Vieille-P...",FRADPE914790181,ORY - Parking P3 SILO - Niveau 2,Parking public,16,Accès libre,Mo-Su 00:00-23:57,False,FRADPE9147901813,2024-10-17,2024-10-17T15:55:47.131000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,2248,support@alizecharge.fr,ADP Groupe,"[2.362554,48.728787]","2 Rue des Transporteurs, 94390 Paray-Vieille-P...",FRADPE91479018,ORY - Parking P3 SILO - Niveau 2,Parking public,16,Accès libre,Mo-Su 00:00-23:57,False,FRADPE914790188,2024-10-17,2024-10-17T15:55:47.131000+00:00
143,2249,support@alizecharge.fr,ADP Groupe,"[2.362554,48.728787]","2 Rue des Transporteurs, 94390 Paray-Vieille-P...",FRADPE914790188,ORY - Parking P3 SILO - Niveau 6,Parking public,18,Accès libre,Mo-Su 00:00-23:57,False,FRADPE9147901880,2024-10-17,2024-10-17T15:55:47.131000+00:00
144,2250,support@alizecharge.fr,ADP Groupe,"[2.362554,48.728787]","2 Rue des Transporteurs, 94390 Paray-Vieille-P...",FRADPE914790188,ORY - Parking P3 SILO - Niveau 6,Parking public,18,Accès libre,Mo-Su 00:00-23:57,False,FRADPE9147901881,2024-10-17,2024-10-17T15:55:47.131000+00:00
145,2251,support@alizecharge.fr,ADP Groupe,"[2.362554,48.728787]","2 Rue des Transporteurs, 94390 Paray-Vieille-P...",FRADPE914790188,ORY - Parking P3 SILO - Niveau 6,Parking public,18,Accès libre,Mo-Su 00:00-23:57,False,FRADPE9147901882,2024-10-17,2024-10-17T15:55:47.131000+00:00


In [90]:
itinerance_4.loc[itinerance_4['contact_operateur'] == 'support@alizecharge.fr', relations].loc[~itinerance_4['adresse_station - coordonneesXY']][400:700]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
2077,91697,support@alizecharge.fr,Ecocharge77,"[2.848262,48.325367]","Rue De La Roche Du Saut, 77250 VILLECERF",FRS77E77501001,VILLECERF - Rue De La Roche Du Saut,Voirie,2,Accès libre,Mo-Su 00:00-23:57,False,FRS77E775010012,2024-10-17,2024-10-17T15:55:47.131000+00:00
2078,91698,support@alizecharge.fr,Ecocharge77,"[2.830368,48.813721]","Place De La Fontaine, 77174 VILLENEUVE-LE-COMTE",FRS77E77508001,VILLENEUVE-LE-COMTE - Place De La Fontaine,Voirie,2,Accès libre,Mo-Su 00:00-23:57,False,FRS77E775080011,2024-10-17,2024-10-17T15:55:47.131000+00:00
2079,91699,support@alizecharge.fr,Ecocharge77,"[2.830368,48.813721]","Place De La Fontaine, 77174 VILLENEUVE-LE-COMTE",FRS77E77508001,VILLENEUVE-LE-COMTE - Place De La Fontaine,Voirie,2,Accès libre,Mo-Su 00:00-23:57,False,FRS77E775080012,2024-10-17,2024-10-17T15:55:47.131000+00:00
2080,91700,support@alizecharge.fr,Ecocharge77,"[3.343021,48.861053]","Rue Du Pont, 77510 VILLENEUVE-SUR-BELLOT",FRS77E77512001,VILLENEUVE-SUR-BELLOT - Rue Du Pont,Voirie,2,Accès libre,Mo-Su 00:00-23:57,False,FRS77E775120011,2024-10-17,2024-10-17T15:55:47.131000+00:00
2081,91701,support@alizecharge.fr,Ecocharge77,"[3.343021,48.861053]","Rue Du Pont, 77510 VILLENEUVE-SUR-BELLOT",FRS77E77512001,VILLENEUVE-SUR-BELLOT - Rue Du Pont,Voirie,2,Accès libre,Mo-Su 00:00-23:57,False,FRS77E775120012,2024-10-17,2024-10-17T15:55:47.131000+00:00
2082,91702,support@alizecharge.fr,Ecocharge77,"[2.895756,48.342731]","Place De L'Église, 77130 VILLE-SAINT-JACQUES",FRS77E77516001,VILLE-SAINT-JACQUES - Place De L'Église,Voirie,2,Accès libre,Mo-Su 00:00-23:57,False,FRS77E775160011,2024-10-17,2024-10-17T15:55:47.131000+00:00
2083,91703,support@alizecharge.fr,Ecocharge77,"[2.895756,48.342731]","Place De L'Église, 77130 VILLE-SAINT-JACQUES",FRS77E77516001,VILLE-SAINT-JACQUES - Place De L'Église,Voirie,2,Accès libre,Mo-Su 00:00-23:57,False,FRS77E775160012,2024-10-17,2024-10-17T15:55:47.131000+00:00
2084,91704,support@alizecharge.fr,Ecocharge77,"[2.672179,48.900551]","Place De Bordeaux, 77410 VILLEVAUDE",FRS77E77517001,VILLEVAUDE - Place De Bordeaux,Voirie,2,Accès libre,Mo-Su 00:00-23:57,False,FRS77E775170011,2024-10-17,2024-10-17T15:55:47.131000+00:00
2085,91705,support@alizecharge.fr,Ecocharge77,"[2.672179,48.900551]","Place De Bordeaux, 77410 VILLEVAUDE",FRS77E77517001,VILLEVAUDE - Place De Bordeaux,Voirie,2,Accès libre,Mo-Su 00:00-23:57,False,FRS77E775170012,2024-10-17,2024-10-17T15:55:47.131000+00:00
2086,91706,support@alizecharge.fr,Ecocharge77,"[2.599086,48.494175]","Rue Cambot, 77190 VILLIERS-EN-BIERE",FRS77E77518001,VILLIERS-EN-BIERE - Rue Cambot,Voirie,2,Accès libre,Mo-Su 00:00-23:57,False,FRS77E775180011,2024-10-17,2024-10-17T15:55:47.131000+00:00


In [100]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[2.969121,48.282539]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
2098,91718,support@alizecharge.fr,Ecocharge77,"[2.969121,48.282539]","Rue Grande, 77940 VOULX",FRS77E77531001,VOULX - Rue Grande,Voirie,2,Accès libre,Mo-Su 00:00-23:57,False,FRS77E775310011,2024-10-17,2024-10-17T15:55:47.131000+00:00
2099,91719,support@alizecharge.fr,Ecocharge77,"[2.969121,48.282539]","Rue Grande, 77940 VOULX",FRS77E77531001,VOULX - Rue Grande,Voirie,2,Accès libre,Mo-Su 00:00-23:57,False,FRS77E775310012,2024-10-17,2024-10-17T15:55:47.131000+00:00
2216,91870,a.buffler@bouygues-es.com,Ecocharge77,"[2.969121,48.282539]",Rue Grande,FRS77P77531A,Voulx - Grande rue,Voirie,2,Accès libre,24/7,false,FRS77EEC7716711,2024-10-02,2024-10-02T12:15:56.201000+00:00
2217,91871,a.buffler@bouygues-es.com,Ecocharge77,"[2.969121,48.282539]",Rue Grande,FRS77P77531A,Voulx - Grande rue,Voirie,2,Accès libre,24/7,false,FRS77EEC7716712,2024-10-02,2024-10-02T12:15:56.201000+00:00


### Defauts operateur IZIVIA
Les défauts au nombre de 280 sont liés à une incohérence entre adresse et coordonnées (une même coordonnée a plusieurs adresses différentes). Ce défaut est lié soit à des erreurs de saisie, soit à des ajouts (ou remplacements) ultérieurs de stations à une même localisation.

Les stations Izivia en erreur ont un id_station_itinerance identique à l'id_pdc_itinerance, ce qui se traduit par des adresses multiples pour une même coordonnée.

Par exemple, sur un parking [-0.530598, 47.398424], on trouve 17 stations avec la même coordonnée, le même identifiant entre point de recharge et station et un nombre de PdC indiqué entre 1 et 3.

In [51]:
itinerance_4.loc[itinerance_4['contact_operateur'] == 'sav@izivia.com', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
273,14588,sav@izivia.com,IZIVIA EXPRESS,"[-0.530598, 47.398424]",26 RUE VALENTIN DES ORMEAUX 49610 MURS-ERIGNE,FRE04POAZS24711,HYPER U - MURS-ERIGNE,Voirie,2,Accès libre,24/7,false,FRE04EOAZS24711,2024-10-23,2024-10-23T08:29:37.424000+00:00
274,14589,sav@izivia.com,IZIVIA EXPRESS,"[-0.530598, 47.398424]",26 RUE VALENTIN DES ORMEAUX 49610 MURS-ERIGNE,FRE04POAZS24712,HYPER U - MURS-ERIGNE,Voirie,2,Accès libre,24/7,false,FRE04EOAZS24712,2024-10-23,2024-10-23T08:29:37.424000+00:00
275,14590,sav@izivia.com,IZIVIA EXPRESS,"[-0.530598, 47.398424]",26 RUE VALENTIN DES ORMEAUX 49610 MURS-ERIGNE,FRE04POAZS24721,HYPER U - MURS-ERIGNE,Voirie,3,Accès libre,24/7,false,FRE04EOAZS24721,2024-10-23,2024-10-23T08:29:37.424000+00:00
276,14591,sav@izivia.com,IZIVIA EXPRESS,"[-0.530598, 47.398424]",26 RUE VALENTIN DES ORMEAUX 49610 MURS-ERIGNE,FRE04POAZS24722,HYPER U - MURS-ERIGNE,Voirie,3,Accès libre,24/7,false,FRE04EOAZS24722,2024-10-23,2024-10-23T08:29:37.424000+00:00
277,14592,sav@izivia.com,IZIVIA EXPRESS,"[-0.530598, 47.398424]",26 RUE VALENTIN DES ORMEAUX 49610 MURS-ERIGNE,FRE04POAZS24723,HYPER U - MURS-ERIGNE,Voirie,3,Accès libre,24/7,false,FRE04EOAZS24723,2024-10-23,2024-10-23T08:29:37.424000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2915,107005,sav@izivia.com,UNIBAIL,"[2.523685, 48.9908]",CENTRE COMMERCIAL DAEROVILLE 95700 ROISSY-EN-F...,FRURWPUNIB1622,AEROVILLE - TERMINAL COOK,Voirie,2,Accès libre,24/7,false,FRURWEUNIB1622,2024-10-23,2024-10-23T08:29:35.808000+00:00
2916,107006,sav@izivia.com,UNIBAIL,"[2.523685, 48.9908]",CENTRE COMMERCIAL DAEROVILLE 95700 ROISSY-EN-F...,FRURWPUNIB1631,AEROVILLE - TERMINAL COOK,Voirie,2,Accès libre,24/7,false,FRURWEUNIB1631,2024-10-23,2024-10-23T08:29:35.808000+00:00
2917,107007,sav@izivia.com,UNIBAIL,"[2.523685, 48.9908]",CENTRE COMMERCIAL DAEROVILLE 95700 ROISSY-EN-F...,FRURWPUNIB1632,AEROVILLE - TERMINAL COOK,Voirie,2,Accès libre,24/7,false,FRURWEUNIB1632,2024-10-23,2024-10-23T08:29:35.808000+00:00
2918,107008,sav@izivia.com,UNIBAIL,"[2.523685, 48.9908]",CENTRE COMMERCIAL AEROVILLE 95700 ROISSY-EN-FR...,FRURWPUNIB1711,AEROVILLE - TOKYO,Voirie,2,Accès libre,24/7,false,FRURWEUNIB1711,2024-10-23,2024-10-23T08:29:35.808000+00:00


In [52]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[-0.530598, 47.398424]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
273,14588,sav@izivia.com,IZIVIA EXPRESS,"[-0.530598, 47.398424]",26 RUE VALENTIN DES ORMEAUX 49610 MURS-ERIGNE,FRE04POAZS24711,HYPER U - MURS-ERIGNE,Voirie,2,Accès libre,24/7,false,FRE04EOAZS24711,2024-10-23,2024-10-23T08:29:37.424000+00:00
274,14589,sav@izivia.com,IZIVIA EXPRESS,"[-0.530598, 47.398424]",26 RUE VALENTIN DES ORMEAUX 49610 MURS-ERIGNE,FRE04POAZS24712,HYPER U - MURS-ERIGNE,Voirie,2,Accès libre,24/7,false,FRE04EOAZS24712,2024-10-23,2024-10-23T08:29:37.424000+00:00
275,14590,sav@izivia.com,IZIVIA EXPRESS,"[-0.530598, 47.398424]",26 RUE VALENTIN DES ORMEAUX 49610 MURS-ERIGNE,FRE04POAZS24721,HYPER U - MURS-ERIGNE,Voirie,3,Accès libre,24/7,false,FRE04EOAZS24721,2024-10-23,2024-10-23T08:29:37.424000+00:00
276,14591,sav@izivia.com,IZIVIA EXPRESS,"[-0.530598, 47.398424]",26 RUE VALENTIN DES ORMEAUX 49610 MURS-ERIGNE,FRE04POAZS24722,HYPER U - MURS-ERIGNE,Voirie,3,Accès libre,24/7,false,FRE04EOAZS24722,2024-10-23,2024-10-23T08:29:37.424000+00:00
277,14592,sav@izivia.com,IZIVIA EXPRESS,"[-0.530598, 47.398424]",26 RUE VALENTIN DES ORMEAUX 49610 MURS-ERIGNE,FRE04POAZS24723,HYPER U - MURS-ERIGNE,Voirie,3,Accès libre,24/7,false,FRE04EOAZS24723,2024-10-23,2024-10-23T08:29:37.424000+00:00
1368,64363,sav@izivia.com,SYSTEME U,"[-0.530598, 47.398424]",RUE VALENTIN DES ORMEAUX 49610 MURS-ERIGNE,FROTHPOTHR372101,HYPER U - MURS-ERIGNE,Voirie,1,Accès libre,24/7,false,FROTHEOTHR372101,2024-10-23,2024-10-23T08:30:05.280000+00:00
1369,64364,sav@izivia.com,SYSTEME U,"[-0.530598, 47.398424]",RUE VALENTIN DES ORMEAUX 49610 MURS-ERIGNE,FROTHPOTHR37211,HYPER U - MURS-ERIGNE,Voirie,1,Accès libre,24/7,false,FROTHEOTHR37211,2024-10-23,2024-10-23T08:30:05.280000+00:00
1370,64365,sav@izivia.com,SYSTEME U,"[-0.530598, 47.398424]",RUE VALENTIN DES ORMEAUX 49610 MURS-ERIGNE,FROTHPOTHR372111,HYPER U - MURS-ERIGNE,Voirie,1,Accès libre,24/7,false,FROTHEOTHR372111,2024-10-23,2024-10-23T08:30:05.280000+00:00
1371,64366,sav@izivia.com,SYSTEME U,"[-0.530598, 47.398424]",RUE VALENTIN DES ORMEAUX 49610 MURS-ERIGNE,FROTHPOTHR372121,HYPER U - MURS-ERIGNE,Voirie,1,Accès libre,24/7,false,FROTHEOTHR372121,2024-10-23,2024-10-23T08:30:05.280000+00:00
1372,64367,sav@izivia.com,SYSTEME U,"[-0.530598, 47.398424]",RUE VALENTIN DES ORMEAUX 49610 MURS-ERIGNE,FROTHPOTHR37221,HYPER U - MURS-ERIGNE,Voirie,1,Accès libre,24/7,false,FROTHEOTHR37221,2024-10-23,2024-10-23T08:30:05.280000+00:00


### Defauts operateur IONITY
Les défauts (220 points de recharge) sont liés à une incohérence entre adresse et coordonnées (une même coordonnée a plusieurs adresses différentes). Ce défaut est lié soit à des erreurs de saisie, soit à des ajouts (ou remplacements) ultérieurs de stations à une même localisation.

Les stations Ionity en erreur ont un id_station_itinerance identique à l'id_pdc_itinerance ainsi qu'un ancien pdc de regroupement par station avec un nom d'enseigne différent (IONITY GMBH), ce qui se traduit par des adresses multiples pour une même coordonnée.

Par exemple, sur l'aire de Mornas [4.732409, 44.194832], on trouve 26 stations avec la même coordonnée, le même identifiant entre point de recharge et station et un nombre de PdC indiqué de 23 pour chaque station.

In [53]:
itinerance_4.loc[itinerance_4['contact_operateur'] == 'info@ionity.eu', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
623,42630,info@ionity.eu,IONITY GMBH,"[1.580008, 48.468782]","Aire de Chartres-Gasville, A11 - 28300 Gasvill...",FRIONE126400,IONITY Chartres Gasville,Station dédiée à la recharge rapide,5,Accès libre,24/7,FALSE,FRIONE1264,2023-03-21,2024-01-19T07:47:22.735000+00:00
624,42635,info@ionity.eu,IONITY GMBH,"[1.416918, 46.313059]","Aire de Boismandé Ouest, A20 - 87160 Saint-Sul...",FRIONE402000,IONITY Boismandé Ouest,Station dédiée à la recharge rapide,5,Accès libre,24/7,FALSE,FRIONE4020,2023-03-21,2024-01-19T07:47:22.735000+00:00
625,42638,info@ionity.eu,IONITY GMBH,"[0.547145, 49.614324]","Aire de services de Bolleville Est, A29, 76210...",FRIONE402300,IONITY Bolleville Est,Station dédiée à la recharge rapide,3,Accès libre,24/7,FALSE,FRIONE4023,2023-03-21,2024-01-19T07:47:22.735000+00:00
626,42642,info@ionity.eu,IONITY GMBH,"[-1.420473, 43.586775]",Aire de Lacq Sud AUDEJOS A64-64170 Lacq,FRIONE402700,IONITY Lacq Audéjos Nord,Station dédiée à la recharge rapide,5,Accès libre,24/7,FALSE,FRIONE4027,2023-03-21,2024-01-19T07:47:22.735000+00:00
627,42648,info@ionity.eu,IONITY GMBH,"[6.623059, 45.196766]","Autoroute de la Maurienne, A43 - Autoport Du F...",FRIONE403700,IONITY Freney,Station dédiée à la recharge rapide,5,Accès libre,24/7,FALSE,FRIONE4037,2023-03-21,2024-01-19T07:47:22.735000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
836,43511,info@ionity.eu,Pech Montat Ouest,"[1.525098, 45.030878]","Aire de service de Pech Montat Ouest, A20, 466...",FRIOYE433403,Pech Montat Ouest,Voirie,7,Accès libre,24/7,false,FRIOYE433403,2024-10-14,2024-10-24T01:00:12.876000+00:00
837,43512,info@ionity.eu,Pech Montat Ouest,"[1.525098, 45.030878]","Aire de service de Pech Montat Ouest, A20, 466...",FRIOYE433404,Pech Montat Ouest,Voirie,7,Accès libre,24/7,false,FRIOYE433404,2024-10-14,2024-10-24T01:00:12.876000+00:00
838,43513,info@ionity.eu,Pech Montat Ouest,"[1.525098, 45.030878]","Aire de service de Pech Montat Ouest, A20, 466...",FRIOYE433451,Pech Montat Ouest,Voirie,7,Accès libre,24/7,false,FRIOYE433451,2024-10-14,2024-10-24T01:00:12.876000+00:00
839,43514,info@ionity.eu,Pech Montat Ouest,"[1.525098, 45.030878]","Aire de service de Pech Montat Ouest, A20, 466...",FRIOYE433452,Pech Montat Ouest,Voirie,7,Accès libre,24/7,false,FRIOYE433452,2024-10-14,2024-10-24T01:00:12.876000+00:00


In [54]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[1.580008, 48.468782]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
623,42630,info@ionity.eu,IONITY GMBH,"[1.580008, 48.468782]","Aire de Chartres-Gasville, A11 - 28300 Gasvill...",FRIONE126400,IONITY Chartres Gasville,Station dédiée à la recharge rapide,5,Accès libre,24/7,FALSE,FRIONE1264,2023-03-21,2024-01-19T07:47:22.735000+00:00
645,42773,info@ionity.eu,Chartres Gasville,"[1.580008, 48.468782]","Chartres-Gasville A11, 28300 Gasville-Oisème",FRIOYE126401,Chartres Gasville,Voirie,7,Accès libre,24/7,false,FRIOYE126401,2024-10-23,2024-10-24T01:00:12.876000+00:00
646,42774,info@ionity.eu,Chartres Gasville,"[1.580008, 48.468782]","Chartres-Gasville A11, 28300 Gasville-Oisème",FRIOYE126402,Chartres Gasville,Voirie,7,Accès libre,24/7,false,FRIOYE126402,2024-10-23,2024-10-24T01:00:12.876000+00:00
647,42775,info@ionity.eu,Chartres Gasville,"[1.580008, 48.468782]","Chartres-Gasville A11, 28300 Gasville-Oisème",FRIOYE126404,Chartres Gasville,Voirie,7,Accès libre,24/7,false,FRIOYE126404,2024-10-23,2024-10-24T01:00:12.876000+00:00
648,42776,info@ionity.eu,Chartres Gasville,"[1.580008, 48.468782]","Chartres-Gasville A11, 28300 Gasville-Oisème",FRIOYE126405,Chartres Gasville,Voirie,7,Accès libre,24/7,false,FRIOYE126405,2024-10-23,2024-10-24T01:00:12.876000+00:00
649,42777,info@ionity.eu,Chartres Gasville,"[1.580008, 48.468782]","Chartres-Gasville A11, 28300 Gasville-Oisème",FRIOYE126451,Chartres Gasville,Voirie,7,Accès libre,24/7,false,FRIOYE126451,2024-10-23,2024-10-24T01:00:12.876000+00:00
650,42778,info@ionity.eu,Chartres Gasville,"[1.580008, 48.468782]","Chartres-Gasville A11, 28300 Gasville-Oisème",FRIOYE126452,Chartres Gasville,Voirie,7,Accès libre,24/7,false,FRIOYE126452,2024-10-23,2024-10-24T01:00:12.876000+00:00
651,42779,info@ionity.eu,Chartres Gasville,"[1.580008, 48.468782]","Chartres-Gasville A11, 28300 Gasville-Oisème",FRIOYE126453,Chartres Gasville,Voirie,7,Accès libre,24/7,false,FRIOYE126453,2024-10-23,2024-10-24T01:00:12.876000+00:00


In [104]:
# example [1.580008, 48.468782]
localisation = itinerance_4.loc[itinerance_4.coordonneesXY == '[1.580008, 48.468782]']
adresses = list(localisation.drop_duplicates('adresse_station')['adresse_station'])
print(adresses)

['Aire de Chartres-Gasville, A11 - 28300 Gasville-Oiséme', 'Chartres-Gasville A11, 28300 Gasville-Oisème']


In [56]:
itinerance_4.loc[itinerance_4['contact_operateur'] == 'info@ionity.eu', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
623,42630,info@ionity.eu,IONITY GMBH,"[1.580008, 48.468782]","Aire de Chartres-Gasville, A11 - 28300 Gasvill...",FRIONE126400,IONITY Chartres Gasville,Station dédiée à la recharge rapide,5,Accès libre,24/7,FALSE,FRIONE1264,2023-03-21,2024-01-19T07:47:22.735000+00:00
624,42635,info@ionity.eu,IONITY GMBH,"[1.416918, 46.313059]","Aire de Boismandé Ouest, A20 - 87160 Saint-Sul...",FRIONE402000,IONITY Boismandé Ouest,Station dédiée à la recharge rapide,5,Accès libre,24/7,FALSE,FRIONE4020,2023-03-21,2024-01-19T07:47:22.735000+00:00
625,42638,info@ionity.eu,IONITY GMBH,"[0.547145, 49.614324]","Aire de services de Bolleville Est, A29, 76210...",FRIONE402300,IONITY Bolleville Est,Station dédiée à la recharge rapide,3,Accès libre,24/7,FALSE,FRIONE4023,2023-03-21,2024-01-19T07:47:22.735000+00:00
626,42642,info@ionity.eu,IONITY GMBH,"[-1.420473, 43.586775]",Aire de Lacq Sud AUDEJOS A64-64170 Lacq,FRIONE402700,IONITY Lacq Audéjos Nord,Station dédiée à la recharge rapide,5,Accès libre,24/7,FALSE,FRIONE4027,2023-03-21,2024-01-19T07:47:22.735000+00:00
627,42648,info@ionity.eu,IONITY GMBH,"[6.623059, 45.196766]","Autoroute de la Maurienne, A43 - Autoport Du F...",FRIONE403700,IONITY Freney,Station dédiée à la recharge rapide,5,Accès libre,24/7,FALSE,FRIONE4037,2023-03-21,2024-01-19T07:47:22.735000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
836,43511,info@ionity.eu,Pech Montat Ouest,"[1.525098, 45.030878]","Aire de service de Pech Montat Ouest, A20, 466...",FRIOYE433403,Pech Montat Ouest,Voirie,7,Accès libre,24/7,false,FRIOYE433403,2024-10-14,2024-10-24T01:00:12.876000+00:00
837,43512,info@ionity.eu,Pech Montat Ouest,"[1.525098, 45.030878]","Aire de service de Pech Montat Ouest, A20, 466...",FRIOYE433404,Pech Montat Ouest,Voirie,7,Accès libre,24/7,false,FRIOYE433404,2024-10-14,2024-10-24T01:00:12.876000+00:00
838,43513,info@ionity.eu,Pech Montat Ouest,"[1.525098, 45.030878]","Aire de service de Pech Montat Ouest, A20, 466...",FRIOYE433451,Pech Montat Ouest,Voirie,7,Accès libre,24/7,false,FRIOYE433451,2024-10-14,2024-10-24T01:00:12.876000+00:00
839,43514,info@ionity.eu,Pech Montat Ouest,"[1.525098, 45.030878]","Aire de service de Pech Montat Ouest, A20, 466...",FRIOYE433452,Pech Montat Ouest,Voirie,7,Accès libre,24/7,false,FRIOYE433452,2024-10-14,2024-10-24T01:00:12.876000+00:00


### Defauts operateur BOUYGUES
Les défauts (286 points de recharge) sont liés à une incohérence entre adresse et coordonnées (une même coordonnée a plusieurs adresses différentes).

Les défauts sont majoritairement associés à l'enseigne Ecocharge77 pour laquelle on trouve des stations en doublon entre Bouygues et AlizeCharge. Ce défaut semble être lié à un transfert de bornes d'un opérateur à un autre sans que les anciennes stations aient été supprimées.

Par exemple, pour la coordonnées [3.438183,48.694653], on trouve quatre points de recharge associés à deux stations (une est opérée par Bouygues et l'autre par AlizeCharge).

In [59]:
itinerance_4.loc[itinerance_4['contact_operateur'] == 'a.buffler@bouygues-es.com', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
2102,91722,a.buffler@bouygues-es.com,Ecocharge77,"[3.438183,48.694653]",route de Brangis,FRS77P77301A,Montceaux-les-Provins,Voirie,2,Accès libre,24/7,false,FRS77EEC7710011,2024-10-02,2024-10-02T12:15:56.201000+00:00
2103,91723,a.buffler@bouygues-es.com,Ecocharge77,"[3.438183,48.694653]",route de Brangis,FRS77P77301A,Montceaux-les-Provins,Voirie,2,Accès libre,24/7,false,FRS77EEC7710012,2024-10-02,2024-10-02T12:15:56.201000+00:00
2104,91724,a.buffler@bouygues-es.com,Ecocharge77,"[2.938246,48.675091]",Avenue du général Leclerc,FRS77P77031A,Bernay-Vilbert - Salle des fêtes,Voirie,2,Accès libre,24/7,false,FRS77EEC771011,2024-10-02,2024-10-02T12:15:56.201000+00:00
2105,91727,a.buffler@bouygues-es.com,Ecocharge77,"[2.938246,48.675091]",Avenue du général Leclerc,FRS77P77031A,Bernay-Vilbert - Salle des fêtes,Voirie,2,Accès libre,24/7,false,FRS77EEC771012,2024-10-02,2024-10-02T12:15:56.201000+00:00
2106,91728,a.buffler@bouygues-es.com,Ecocharge77,"[3.427434,48.851772]",rue Brulée,FRS77P77303A,Montdauphin,Voirie,2,Accès libre,24/7,false,FRS77EEC7710211,2024-10-02,2024-10-02T12:15:56.201000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2393,92169,a.buffler@bouygues-es.com,Ecocharge77,"[2.859685,48.926731]",Place Jean Jaurès,FRS77P77276A,Mareuil-les-Meaux,Voirie,2,Accès libre,24/7,false,FRS77EEC779412,2024-10-02,2024-10-02T12:15:56.201000+00:00
2394,92176,a.buffler@bouygues-es.com,Ecocharge77,"[2.850102,48.349255]",Rue Grande,FRS77P77299A,Morêt-Loing-et-Orvanne - Montarlot,Voirie,2,Accès libre,24/7,false,FRS77EEC779811,2024-10-02,2024-10-02T12:15:56.201000+00:00
2395,92177,a.buffler@bouygues-es.com,Ecocharge77,"[2.850102,48.349255]",Rue Grande,FRS77P77299A,Morêt-Loing-et-Orvanne - Montarlot,Voirie,2,Accès libre,24/7,false,FRS77EEC779812,2024-10-02,2024-10-02T12:15:56.201000+00:00
2396,92178,a.buffler@bouygues-es.com,Ecocharge77,"[2.985625,48.941795]",place de la mairie,FRS77P77300A,Montceaux-les-Meaux,Voirie,2,Accès libre,24/7,false,FRS77EEC779911,2024-10-02,2024-10-02T12:15:56.201000+00:00


In [60]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[2.850102,48.349255]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
1984,91554,support@alizecharge.fr,Ecocharge77,"[2.850102,48.349255]","Rue Grande, 77250 MORET-LOING-ORVANNE",FRS77E77316001,MORET-LOING-ORVANNE - Rue Grande,Voirie,2,Accès libre,Mo-Su 00:00-23:57,False,FRS77E773160011,2024-10-17,2024-10-17T15:55:47.131000+00:00
1985,91555,support@alizecharge.fr,Ecocharge77,"[2.850102,48.349255]","Rue Grande, 77250 MORET-LOING-ORVANNE",FRS77E77316001,MORET-LOING-ORVANNE - Rue Grande,Voirie,2,Accès libre,Mo-Su 00:00-23:57,False,FRS77E773160012,2024-10-17,2024-10-17T15:55:47.131000+00:00
2394,92176,a.buffler@bouygues-es.com,Ecocharge77,"[2.850102,48.349255]",Rue Grande,FRS77P77299A,Morêt-Loing-et-Orvanne - Montarlot,Voirie,2,Accès libre,24/7,false,FRS77EEC779811,2024-10-02,2024-10-02T12:15:56.201000+00:00
2395,92177,a.buffler@bouygues-es.com,Ecocharge77,"[2.850102,48.349255]",Rue Grande,FRS77P77299A,Morêt-Loing-et-Orvanne - Montarlot,Voirie,2,Accès libre,24/7,false,FRS77EEC779812,2024-10-02,2024-10-02T12:15:56.201000+00:00


In [61]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[3.438183,48.694653]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
1974,91540,support@alizecharge.fr,Ecocharge77,"[3.438183,48.694653]","Route De Brangis, 77151 MONTCEAUX-LES-PROVINS",FRS77E77301001,MONTCEAUX-LES-PROVINS - Route De Brangis,Voirie,2,Accès libre,Mo-Su 00:00-23:57,False,FRS77E773010011,2024-10-17,2024-10-17T15:55:47.131000+00:00
1975,91541,support@alizecharge.fr,Ecocharge77,"[3.438183,48.694653]","Route De Brangis, 77151 MONTCEAUX-LES-PROVINS",FRS77E77301001,MONTCEAUX-LES-PROVINS - Route De Brangis,Voirie,2,Accès libre,Mo-Su 00:00-23:57,False,FRS77E773010012,2024-10-17,2024-10-17T15:55:47.131000+00:00
2102,91722,a.buffler@bouygues-es.com,Ecocharge77,"[3.438183,48.694653]",route de Brangis,FRS77P77301A,Montceaux-les-Provins,Voirie,2,Accès libre,24/7,false,FRS77EEC7710011,2024-10-02,2024-10-02T12:15:56.201000+00:00
2103,91723,a.buffler@bouygues-es.com,Ecocharge77,"[3.438183,48.694653]",route de Brangis,FRS77P77301A,Montceaux-les-Provins,Voirie,2,Accès libre,24/7,false,FRS77EEC7710012,2024-10-02,2024-10-02T12:15:56.201000+00:00


### Cohérence implantation_station - id_station
- 107 pdc sont liés à une erreur de choix d'implantation (couplé à d'autres erreurs)

In [62]:
itinerance_4.loc[~itinerance_4['implantation_station - id_station_itinerance'], relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
47,2017,support@alizecharge.fr,ADP Groupe,"[2.577444,49.004299]","Route des Peupliers, 77990 Le Mesnil-Amelot",FRADPE77291004,CDG - Parking PEF - Niveau -2 - Allée 14 & 15,Parking public,32,Accès libre,Mo-Su 00:00-23:57,False,FRADPE772910041,2024-10-17,2024-10-17T15:55:47.131000+00:00
48,2028,support@alizecharge.fr,ADP Groupe,"[2.577444,49.004299]","Route des Peupliers, 77990 Le Mesnil-Amelot",FRADPE77291004,CDG - Parking PEF - Niveau -2 - Allée 14 & 15,Parking public,32,Accès libre,Mo-Su 00:00-23:57,False,FRADPE772910042,2024-10-17,2024-10-17T15:55:47.131000+00:00
49,2039,support@alizecharge.fr,ADP Groupe,"[2.577444,49.004299]","Route des Peupliers, 77990 Le Mesnil-Amelot",FRADPE77291004,CDG - Parking PEF - Niveau -2 - Allée 14 & 15,Voirie,32,Accès libre,Mo-Su 00:00-23:57,False,FRADPE772910043,2024-10-17,2024-10-17T15:55:47.131000+00:00
50,2043,support@alizecharge.fr,ADP Groupe,"[2.577444,49.004299]","Route des Peupliers, 77990 Le Mesnil-Amelot",FRADPE77291004,CDG - Parking PEF - Niveau -2 - Allée 14 & 15,Parking public,32,Accès libre,Mo-Su 00:00-23:57,False,FRADPE772910044,2024-10-17,2024-10-17T15:55:47.131000+00:00
51,2044,support@alizecharge.fr,ADP Groupe,"[2.577444,49.004299]","Route des Peupliers, 77990 Le Mesnil-Amelot",FRADPE77291004,CDG - Parking PEF - Niveau -2 - Allée 14 & 15,Parking public,32,Accès libre,Mo-Su 00:00-23:57,False,FRADPE772910045,2024-10-17,2024-10-17T15:55:47.131000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2589,97420,customerservice@shellrecharge.com,Shell Fenioux Est,"[-0.60280679, 45.89253519]","A10 Aire de Fenioux Est, 17350 Fenioux",FRSHEE94,Shell Fenioux Est,Voirie,8,Accès réservé,24/7,false,FRSHEE944,2024-10-23,2024-10-24T01:00:12.876000+00:00
2590,97421,customerservice@shellrecharge.com,Shell Fenioux Est,"[-0.60280679, 45.89253519]","A10 Aire de Fenioux Est, 17350 Fenioux",FRSHEE95,Shell Fenioux Est,Station dédiée à la recharge rapide,8,Accès réservé,24/7,false,FRSHEE951,2024-10-23,2024-10-24T01:00:12.876000+00:00
2591,97422,customerservice@shellrecharge.com,Shell Fenioux Est,"[-0.60280679, 45.89253519]","A10 Aire de Fenioux Est, 17350 Fenioux",FRSHEE95,Shell Fenioux Est,Station dédiée à la recharge rapide,8,Accès réservé,24/7,false,FRSHEE952,2024-10-23,2024-10-24T01:00:12.876000+00:00
2592,97423,customerservice@shellrecharge.com,Shell Fenioux Est,"[-0.60280679, 45.89253519]","A10 Aire de Fenioux Est, 17350 Fenioux",FRSHEE95,Shell Fenioux Est,Station dédiée à la recharge rapide,8,Accès réservé,24/7,false,FRSHEE953,2024-10-23,2024-10-24T01:00:12.876000+00:00


### Cohérence nom_station - id_station
- 210 pdc sont associés à une station avec un nom non cohérent

In [63]:
itinerance_4.loc[~itinerance_4['nom_station - id_station_itinerance'], relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
66,2172,support@alizecharge.fr,ADP Groupe,"[2.362554,48.728787]","2 Rue des Transporteurs, 94390 Paray-Vieille-P...",FRADPE914790181,ORY - Parking P3 SILO - Niveau 2,Parking public,16,Accès libre,Mo-Su 00:00-23:57,False,FRADPE9147901810,2024-10-17,2024-10-17T15:55:47.131000+00:00
67,2173,support@alizecharge.fr,ADP Groupe,"[2.362554,48.728787]","2 Rue des Transporteurs, 94390 Paray-Vieille-P...",FRADPE914790181,ORY - Parking P3 SILO - Niveau 2,Parking public,16,Accès libre,Mo-Su 00:00-23:57,False,FRADPE9147901811,2024-10-17,2024-10-17T15:55:47.131000+00:00
68,2174,support@alizecharge.fr,ADP Groupe,"[2.362554,48.728787]","2 Rue des Transporteurs, 94390 Paray-Vieille-P...",FRADPE914790181,ORY - Parking P3 SILO - Niveau 2,Parking public,16,Accès libre,Mo-Su 00:00-23:57,False,FRADPE9147901812,2024-10-17,2024-10-17T15:55:47.131000+00:00
69,2175,support@alizecharge.fr,ADP Groupe,"[2.362554,48.728787]","2 Rue des Transporteurs, 94390 Paray-Vieille-P...",FRADPE914790181,ORY - Parking P3 SILO - Niveau 2,Parking public,16,Accès libre,Mo-Su 00:00-23:57,False,FRADPE9147901813,2024-10-17,2024-10-17T15:55:47.131000+00:00
70,2176,support@alizecharge.fr,ADP Groupe,"[2.362554,48.728787]","2 Rue des Transporteurs, 94390 Paray-Vieille-P...",FRADPE914790181,ORY - Parking P3 SILO - Niveau 2,Parking public,16,Accès libre,Mo-Su 00:00-23:57,False,FRADPE9147901814,2024-10-17,2024-10-17T15:55:47.131000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3381,111859,support@greenflux.com,ENGIE Vianeo,"[0.048634, 44.293855]",CENTER PARCS LANDES DE GASCOGNE - Parking H - ...,FRVIAP123020,CENTER PARCS LANDES DE GASCOGNE - Parking H,Station dédiée à la recharge rapide,1,Accès libre,24/7,false,FRVIAE20123020022,2024-10-02,2024-10-08T15:23:52.217000+00:00
3382,111860,support@greenflux.com,ENGIE Vianeo,"[0.048634, 44.293855]",CENTER PARCS LANDES DE GASCOGNE - Parking H - ...,FRVIAP123020,CENTER PARCS LANDES DE GASCOGNE - Parking H,Station dédiée à la recharge rapide,1,Accès libre,24/7,false,FRVIAE20123020023,2024-10-02,2024-10-08T15:23:52.217000+00:00
3383,111861,support@greenflux.com,ENGIE Vianeo,"[0.048634, 44.293855]",CENTER PARCS LANDES DE GASCOGNE - Parking H - ...,FRVIAP123020,CENTER PARCS LANDES DE GASCOGNE - Parking H,Station dédiée à la recharge rapide,1,Accès libre,24/7,false,FRVIAE20123020024,2024-10-02,2024-10-08T15:23:52.217000+00:00
3384,111862,support@greenflux.com,ENGIE Vianeo,"[0.048634, 44.293855]",CENTER PARCS LANDES DE GASCOGNE - Parking H - ...,FRVIAP123020,CENTER PARCS LANDES DE GASCOGNE - Parking H,Station dédiée à la recharge rapide,1,Accès libre,24/7,false,FRVIAE20123020025,2024-10-02,2024-10-08T15:23:52.217000+00:00


### Coherence adresse - coordonnees
- 2337 pdc ont une adresse non cohérente avec les coordonnées géographiques

In [64]:
itinerance_4.loc[~itinerance_4['adresse_station - coordonneesXY'], relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
20,336,exploitation@r3-charge.fr,R3,"[3.472424, 50.379911]","400 Rue Michel Chasles, 59494 Petite-Forêt",FR3R3P89616641,R3 - Petite Forêt,Parking privé à usage public,5,Accès libre,Mo-Su 07:00-22:00,false,FR3R3E10000849681,2024-09-30,2024-09-30T08:00:36.174000+00:00
21,337,exploitation@r3-charge.fr,R3,"[3.472424, 50.379911]","401 Rue Michel Chasles, 59494 Petite-Forêt",FR3R3P89616641,R3 - Petite Forêt,Parking privé à usage public,5,Accès libre,Mo-Su 07:00-22:01,false,FR3R3E10000849682,2024-09-30,2024-09-30T08:00:36.174000+00:00
22,338,exploitation@r3-charge.fr,R3,"[1.613217, 50.485562]",708 Av. François Godin Bis 62780 Cucq,FR3R3P89629589,R3 - Cucq,Parking privé à usage public,7,Accès libre,24/7,false,FR3R3E10000849691,2024-09-30,2024-09-30T08:00:36.174000+00:00
23,339,exploitation@r3-charge.fr,R3,"[1.613217, 50.485562]",709 Av. François Godin Bis 62780 Cucq,FR3R3P89629589,R3 - Cucq,Parking privé à usage public,7,Accès libre,24/7,false,FR3R3E10000849692,2024-09-30,2024-09-30T08:00:36.174000+00:00
24,364,exploitation@r3-charge.fr,R3,"[1.613217, 50.485562]",713 Av. François Godin Bis 62780 Cucq,FR3R3P89629589,R3 - Cucq,Parking privé à usage public,7,Accès libre,24/7,false,FR3R3E10000849851,2024-09-30,2024-09-30T08:00:36.174000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3511,115072,advenir@zeborne.com,GARAGE NELLO CHELLI - DIJON,"[5.065799, 47.354481]",23 RUE DES ARDENNES 21000 Dijon,FRZTLE22AC55087,Toyota - Dijon - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 07:45-12:00,Mo-Fr 13:45-19:00,Sat 09:00-...",false,FRZTLE22AC55087,2021-02-19,2024-01-19T07:46:24.141000+00:00
3512,115083,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57332,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57332,2021-04-08,2024-01-19T07:46:23.908000+00:00
3513,115084,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57333,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57333,2021-04-08,2024-01-19T07:46:23.908000+00:00
3514,115085,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57334,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57334,2021-04-08,2024-01-19T07:46:23.908000+00:00


In [65]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[5.2592028, 46.1746523]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
3512,115083,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57332,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57332,2021-04-08,2024-01-19T07:46:23.908000+00:00
3513,115084,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57333,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57333,2021-04-08,2024-01-19T07:46:23.908000+00:00
3514,115085,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57334,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57334,2021-04-08,2024-01-19T07:46:23.908000+00:00
3515,115086,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57335,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57335,2021-04-08,2024-01-19T07:46:23.908000+00:00


In [66]:
itinerance_4.loc[itinerance_4.adresse_station == 'RN 75 ZI DE NOIREFONTAINE 01250 Montagnat', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
3512,115083,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57332,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57332,2021-04-08,2024-01-19T07:46:23.908000+00:00
3514,115085,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57334,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57334,2021-04-08,2024-01-19T07:46:23.908000+00:00


In [67]:
itinerance_4.loc[itinerance_4.adresse_station == ' RN 75 ZI DE NOIREFONTAINE 01250 Montagnat', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
3513,115084,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57333,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57333,2021-04-08,2024-01-19T07:46:23.908000+00:00
3515,115086,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57335,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57335,2021-04-08,2024-01-19T07:46:23.908000+00:00


### Cohérence station - coordonnées
- 218 pdc sont associés à des stations avec plusieurs coordonnées (majorité AlizeCharge)

In [68]:
itinerance_4.loc[~itinerance_4['coordonneesXY - id_station_itinerance'], relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
1,173,acelec@acelec-france.com,Camaïeu France,"[3.207306,50.684918]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073E0HKH51125,2022-07-21,2024-01-19T07:46:04.014000+00:00
2,174,acelec@acelec-france.com,Camaïeu France,"[3.207246,50.68494]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073E8OV45243,2022-07-21,2024-01-19T07:46:04.014000+00:00
3,175,acelec@acelec-france.com,Camaïeu France,"[3.207124,50.684984]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EBEQA5957,2022-07-21,2024-01-19T07:46:04.014000+00:00
4,176,acelec@acelec-france.com,Camaïeu France,"[3.20737,50.684898]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073ECC1C5530,2022-07-21,2024-01-19T07:46:04.014000+00:00
5,177,acelec@acelec-france.com,HOTEL KYRIAD PARIS BEZONS,"[2.212336,48.931356]",80 Av. Gabriel Péri 95870 BEZONS,FR073PKYRIADBEZONS,HOTEL KYRIAD PARIS BEZONS,Parking privé à usage public,1,Accès libre,Mo-Fr 08:00-19:01,false,FR073EJJYD51230,2022-07-21,2024-01-19T07:46:04.014000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3381,111859,support@greenflux.com,ENGIE Vianeo,"[0.048634, 44.293855]",CENTER PARCS LANDES DE GASCOGNE - Parking H - ...,FRVIAP123020,CENTER PARCS LANDES DE GASCOGNE - Parking H,Station dédiée à la recharge rapide,1,Accès libre,24/7,false,FRVIAE20123020022,2024-10-02,2024-10-08T15:23:52.217000+00:00
3382,111860,support@greenflux.com,ENGIE Vianeo,"[0.048634, 44.293855]",CENTER PARCS LANDES DE GASCOGNE - Parking H - ...,FRVIAP123020,CENTER PARCS LANDES DE GASCOGNE - Parking H,Station dédiée à la recharge rapide,1,Accès libre,24/7,false,FRVIAE20123020023,2024-10-02,2024-10-08T15:23:52.217000+00:00
3383,111861,support@greenflux.com,ENGIE Vianeo,"[0.048634, 44.293855]",CENTER PARCS LANDES DE GASCOGNE - Parking H - ...,FRVIAP123020,CENTER PARCS LANDES DE GASCOGNE - Parking H,Station dédiée à la recharge rapide,1,Accès libre,24/7,false,FRVIAE20123020024,2024-10-02,2024-10-08T15:23:52.217000+00:00
3384,111862,support@greenflux.com,ENGIE Vianeo,"[0.048634, 44.293855]",CENTER PARCS LANDES DE GASCOGNE - Parking H - ...,FRVIAP123020,CENTER PARCS LANDES DE GASCOGNE - Parking H,Station dédiée à la recharge rapide,1,Accès libre,24/7,false,FRVIAE20123020025,2024-10-02,2024-10-08T15:23:52.217000+00:00


In [69]:
itinerance_5.loc[itinerance_5.id_station_itinerance	 == 'FRTLSE31555040', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
2714,103890,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.4503666,43.6039735]","61 Bd Lazare Carnot, 31000 TOULOUSE",FRTLSE31555040,TOULOUSE - Bd Lazare Carnot,Voirie,1,Accès libre,Mo-Su 00:00-23:57,False,FRTLSE315550401,2024-10-17,2024-10-17T15:55:47.131000+00:00
2715,103891,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.4503666,43.6039735]","61 Bd Lazare Carnot, 31000 TOULOUSE",FRTLSE31555040,TOULOUSE - Bd Lazare Carnot,Voirie,1,Accès libre,Mo-Su 00:00-23:57,False,FRTLSE315550402,2024-10-17,2024-10-17T15:55:47.131000+00:00
2716,103892,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.45026,43.6040701]","63 Bd Lazare Carnot, 31000 TOULOUSE",FRTLSE31555040,TOULOUSE – Station Deux-Roues Lazare Carnot,Voirie,3,Accès libre,Mo-Su 00:00-23:57,False,FRTLSE315550403,2024-10-17,2024-10-17T15:55:47.131000+00:00
2717,103893,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.45026,43.6040701]","63 Bd Lazare Carnot, 31000 TOULOUSE",FRTLSE31555040,TOULOUSE – Station Deux-Roues Lazare Carnot,Voirie,3,Accès libre,Mo-Su 00:00-23:57,False,FRTLSE315550404,2024-10-17,2024-10-17T15:55:47.131000+00:00
2718,103894,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.45026,43.6040701]","63 Bd Lazare Carnot, 31000 TOULOUSE",FRTLSE31555040,TOULOUSE – Station Deux-Roues Lazare Carnot,Voirie,3,Accès libre,Mo-Su 00:00-23:57,False,FRTLSE315550405,2024-10-17,2024-10-17T15:55:47.131000+00:00


In [70]:
itinerance_5.loc[itinerance_5.id_station_itinerance	 == 'FRFR1PEHNKRRPQNR', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified


- exemple : une station avec plusieurs coordonnées -> incompréhension distinction station / pdc

In [71]:
itinerance_5.loc[itinerance_5.id_station_itinerance == 'FR073PCAMAIEUFR', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
1,173,acelec@acelec-france.com,Camaïeu France,"[3.207306,50.684918]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073E0HKH51125,2022-07-21,2024-01-19T07:46:04.014000+00:00
2,174,acelec@acelec-france.com,Camaïeu France,"[3.207246,50.68494]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073E8OV45243,2022-07-21,2024-01-19T07:46:04.014000+00:00
3,175,acelec@acelec-france.com,Camaïeu France,"[3.207124,50.684984]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EBEQA5957,2022-07-21,2024-01-19T07:46:04.014000+00:00
4,176,acelec@acelec-france.com,Camaïeu France,"[3.20737,50.684898]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073ECC1C5530,2022-07-21,2024-01-19T07:46:04.014000+00:00
6,178,acelec@acelec-france.com,Camaïeu France,"[3.207306,50.684918]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EK0BJ522,2022-07-21,2024-01-19T07:46:04.014000+00:00
7,179,acelec@acelec-france.com,Camaïeu France,"[3.207047,50.685013]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EMZLV5283,2022-07-21,2024-01-19T07:46:04.014000+00:00
8,180,acelec@acelec-france.com,Camaïeu France,"[3.20737,50.684898]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EO0AV51091,2022-07-21,2024-01-19T07:46:04.014000+00:00
9,181,acelec@acelec-france.com,Camaïeu France,"[3.207187,50.684964]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EP7QU5233,2022-07-21,2024-01-19T07:46:04.014000+00:00
10,182,acelec@acelec-france.com,Camaïeu France,"[3.206962,50.685049]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EP9LD51287,2022-07-21,2024-01-19T07:46:04.014000+00:00
11,183,acelec@acelec-france.com,Camaïeu France,"[3.207433,50.684876]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EQ1GI51176,2022-07-21,2024-01-19T07:46:04.014000+00:00


- exemple : une station avec plusieurs coordonnées -> regroupement de stations ?

In [72]:
itinerance_5.loc[itinerance_5.id_station_itinerance == 'FR55CPBP514', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified


- exemple : plusieurs stations d'opérateurs différents avec les mêmes coordonnées -> suppression non effectuée lors d'un changement d'opérateur ?

In [73]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[0.08745032, 48.40887156]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified


In [74]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[-4.374469, 48.471878]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified


- exemple : quatre stations avec une localisation identique mais deux adresses différentes + id identique entre station et pdc -> erreur d'adresse et erreur d'id_station ?

In [75]:
itinerance_5.loc[itinerance_5.coordonneesXY	 == '[-0.366184, 43.332508]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified


### Incoherence nbre_pdc
- 583 pdc ont un champ 'nbre_pdc' mal documenté

In [76]:
itinerance_4.loc[~itinerance_4['nbre_pdc - id_station_itinerance'], relations][200:215]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
1036,56665,support@alizecharge.fr,Grenoble-Alpes Métropole,"[5.660848,45.253881]","Rue des Mûriers, 38120 PALLUEL",FRM38E38120001,PALLUEL - Parking Le Fontanil Cornillon (CITIZ),Voirie,1,Accès libre,Mo-Su 00:00-23:57,False,FRM38E381200011,2024-10-17,2024-10-17T15:55:47.131000+00:00
1037,56666,support@alizecharge.fr,Grenoble-Alpes Métropole,"[5.660848,45.253881]","Rue des Mûriers, 38120 FONTANIL-CORNILLON",FRM38E38120001,FONTANIL-CORNILLON - P+R Le Fontanil Cornillon...,Voirie,7,Accès libre,Mo-Su 00:00-23:57,False,FRM38E381200012,2024-10-17,2024-10-17T15:55:47.131000+00:00
1038,56667,support@alizecharge.fr,Grenoble-Alpes Métropole,"[5.660848,45.253881]","Rue des Mûriers, 38120 FONTANIL-CORNILLON",FRM38E38120001,FONTANIL-CORNILLON - P+R Le Fontanil Cornillon...,Voirie,7,Accès libre,Mo-Su 00:00-23:57,False,FRM38E381200013,2024-10-17,2024-10-17T15:55:47.131000+00:00
1039,56668,support@alizecharge.fr,Grenoble-Alpes Métropole,"[5.660848,45.253881]","Rue des Mûriers, 38120 FONTANIL-CORNILLON",FRM38E38120001,FONTANIL-CORNILLON - P+R Le Fontanil Cornillon...,Voirie,7,Accès libre,Mo-Su 00:00-23:57,False,FRM38E381200014,2024-10-17,2024-10-17T15:55:47.131000+00:00
1040,56669,support@alizecharge.fr,Grenoble-Alpes Métropole,"[5.660848,45.253881]","Rue des Mûriers, 38120 FONTANIL-CORNILLON",FRM38E38120001,FONTANIL-CORNILLON - P+R Le Fontanil Cornillon...,Voirie,7,Accès libre,Mo-Su 00:00-23:57,False,FRM38E381200015,2024-10-17,2024-10-17T15:55:47.131000+00:00
1041,56670,support@alizecharge.fr,Grenoble-Alpes Métropole,"[5.660848,45.253881]","Rue des Mûriers, 38120 FONTANIL-CORNILLON",FRM38E38120001,FONTANIL-CORNILLON - P+R Le Fontanil Cornillon...,Voirie,7,Accès libre,Mo-Su 00:00-23:57,False,FRM38E381200016,2024-10-17,2024-10-17T15:55:47.131000+00:00
1042,56671,support@alizecharge.fr,Grenoble-Alpes Métropole,"[5.660848,45.253881]","Rue des Mûriers, 38120 FONTANIL-CORNILLON",FRM38E38120001,FONTANIL-CORNILLON - P+R Le Fontanil Cornillon...,Voirie,7,Accès libre,Mo-Su 00:00-23:57,False,FRM38E381200017,2024-10-17,2024-10-17T15:55:47.131000+00:00
1043,56672,support@alizecharge.fr,Grenoble-Alpes Métropole,"[5.660848,45.253881]","Rue des Mûriers, 38120 FONTANIL-CORNILLON",FRM38E38120001,FONTANIL-CORNILLON - P+R Le Fontanil Cornillon...,Voirie,7,Accès libre,Mo-Su 00:00-23:57,False,FRM38E381200018,2024-10-17,2024-10-17T15:55:47.131000+00:00
1044,56675,support@alizecharge.fr,Grenoble-Alpes Métropole,"[5.719183,45.152774]","Rue de Provence, 38130 ECHIROLLES",FRM38E38151001,ECHIROLLES - Parking Relais Échirolles - Gare,Voirie,5,Accès libre,Mo-Su 00:00-23:57,False,FRM38E381510011,2024-10-17,2024-10-17T15:55:47.131000+00:00
1045,56676,support@alizecharge.fr,Grenoble-Alpes Métropole,"[5.719183,45.152774]","Rue de Provence, 38130 ECHIROLLES",FRM38E38151001,ECHIROLLES - Parking Relais Échirolles - Gare,Voirie,5,Accès libre,Mo-Su 00:00-23:57,False,FRM38E381510012,2024-10-17,2024-10-17T15:55:47.131000+00:00


### coherence station - enseigne
- 5 stations ont plusieurs noms d'enseigne

In [77]:
itinerance_4.loc[~itinerance_4['nom_enseigne - id_station_itinerance'], relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
446,30168,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.014273, 50.615449]",12 Rue Salengro LOOS,FRFR1EAYRU,"Loos, Parking Salengro",Parking public,3,Accès libre,24/7,FALSE,FRFR1EDXS1,2022-12-09,2024-01-19T07:50:45.527000+00:00
447,30169,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.014273, 50.615449]",12 Rue Salengro LOOS,FRFR1EAYRU,"Loos, Parking Salengro",Parking public,3,Accès libre,24/7,FALSE,FRFR1EDXS2,2022-12-09,2024-01-19T07:50:45.527000+00:00
448,30170,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.014273, 50.615449]",12 Rue Salengro LOOS,FRFR1EAYRU,"Loos, Parking Salengro",Parking public,3,Accès libre,24/7,FALSE,FRFR1EDXS3,2022-12-09,2024-01-19T07:50:45.527000+00:00
1015,55160,contact@rirodo.fr,LAST MILES SOLUTIONS,"[5.39, 43.27]",20 Rue Negresko,FRRIRE2804856,PARKING NEGRESKO,Parking privé à usage public,2,Accès libre,24/7,false,FRLMSE1897228,2023-09-06,2024-01-19T07:49:16.739000+00:00
1102,59400,support@mobilize-ps.fr,JRA SAUMUR,"[-0.059981, 47.274681]","\tBd Vallerots, 49400 Saumur",FRMBZEEVHJW,JRA SAUMUR,Parking privé à usage public,1,Accès libre,"Mo-Fr 07:30-12:00,Mo-Fr 14:00-19:00",False,FRMBZEEVHJW1,2024-07-10,2024-10-23T15:12:49.416000+00:00
1103,59401,support@mobilize-ps.fr,JRA Saumur,"[-0.05996536452405448, 47.27462246981463]","Bd Vallerots, 49400 Saumur",FRMBZEEVHJW,JRA Saumur,Parking privé à usage public,1,Accès libre,24/7,False,FRMBZEEVHJW2,2024-07-10,2024-10-23T15:12:49.416000+00:00
1113,59438,support@mobilize-ps.fr,BODEMER BAYEUX,"[-0.700226002831211, 49.27084281571326]","16 Bd Sadi Carnot, 14400 Bayeux",FRMBZESMYQC,BODEMER Bayeux,Parking privé à usage public,1,Accès libre,24/7,False,FRMBZESMYCQ1,2024-06-03,2024-10-23T15:12:49.416000+00:00
1114,59439,support@mobilize-ps.fr,BODEMER - SCAUTO BAYEUX,"[-0.700259, 49.270901]","\t16 Bd Sadi Carnot, 14400 Bayeux",FRMBZESMYQC,\t\tScauto Bayeux,Parking privé à usage public,1,Accès libre,"Mo-Fr 07:30-12:00,Mo-Fr 14:00-19:00",False,FRMBZESMYQC1,2024-06-03,2024-10-23T15:12:49.416000+00:00
1540,78454,contact@rirodo.fr,MONTA,"[5.39, 43.27]",20 Rue NEGRESKO,FRRIRE2804856,PARKING NEGRESKO,Parking privé à usage public,2,Accès libre,24/7,false,FRRIRE2804856,2023-09-06,2024-01-19T07:49:16.739000+00:00


In [78]:
itinerance_init.loc[itinerance_init.id_station_itinerance == 'FRFR1EAYRU', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
29249,29269,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.02,50.61]",rue du colonnel jean colonna d'ornano loos,FRFR1EAYRU,"Loos, Parking de la Gare",Parking public,3,Accès libre,24/7,false,FRFR1EAYRU1,2022-12-25,2024-01-19T07:49:57.766000+00:00
29251,29271,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.02,50.61]",rue du colonnel ornano loos,FRFR1EAYRU,"Loos, Parking de la Gare",Parking public,3,Accès libre,24/7,false,FRFR1EAYRU2,2022-08-25,2024-01-19T07:49:57.766000+00:00
29253,29273,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.02,50.61]",rue du colonnel ornano loos,FRFR1EAYRU,"Loos, Parking de la Gare",Parking public,3,Accès libre,24/7,false,FRFR1EAYRU3,2022-08-25,2024-01-19T07:49:57.766000+00:00
30148,30168,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.014273, 50.615449]",12 Rue Salengro LOOS,FRFR1EAYRU,"Loos, Parking Salengro",Parking public,3,Accès libre,24/7,FALSE,FRFR1EDXS1,2022-12-09,2024-01-19T07:50:45.527000+00:00
30149,30169,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.014273, 50.615449]",12 Rue Salengro LOOS,FRFR1EAYRU,"Loos, Parking Salengro",Parking public,3,Accès libre,24/7,FALSE,FRFR1EDXS2,2022-12-09,2024-01-19T07:50:45.527000+00:00
30150,30170,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.014273, 50.615449]",12 Rue Salengro LOOS,FRFR1EAYRU,"Loos, Parking Salengro",Parking public,3,Accès libre,24/7,FALSE,FRFR1EDXS3,2022-12-09,2024-01-19T07:50:45.527000+00:00


In [79]:
itinerance_init.loc[itinerance_init.nom_enseigne == 'RME Loos (FR-FR1)', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
29249,29269,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.02,50.61]",rue du colonnel jean colonna d'ornano loos,FRFR1EAYRU,"Loos, Parking de la Gare",Parking public,3,Accès libre,24/7,false,FRFR1EAYRU1,2022-12-25,2024-01-19T07:49:57.766000+00:00
29251,29271,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.02,50.61]",rue du colonnel ornano loos,FRFR1EAYRU,"Loos, Parking de la Gare",Parking public,3,Accès libre,24/7,false,FRFR1EAYRU2,2022-08-25,2024-01-19T07:49:57.766000+00:00
30148,30168,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.014273, 50.615449]",12 Rue Salengro LOOS,FRFR1EAYRU,"Loos, Parking Salengro",Parking public,3,Accès libre,24/7,FALSE,FRFR1EDXS1,2022-12-09,2024-01-19T07:50:45.527000+00:00
30149,30169,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.014273, 50.615449]",12 Rue Salengro LOOS,FRFR1EAYRU,"Loos, Parking Salengro",Parking public,3,Accès libre,24/7,FALSE,FRFR1EDXS2,2022-12-09,2024-01-19T07:50:45.527000+00:00
33436,33456,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.009613, 50.617509]",12 Rue Salengro LOOS,FRFR1EPJXS,"Loos, Parking CIL",Parking public,3,Accès libre,24/7,FALSE,FRFR1EPJXS1,2022-10-04,2024-01-19T07:49:31.469000+00:00
33438,33458,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.009613, 50.617509]",12 Rue Salengro LOOS,FRFR1EPJXS,"Loos, Parking CIL",Parking public,3,Accès libre,24/7,FALSE,FRFR1EPJXS2,2022-10-04,2024-01-19T07:49:31.469000+00:00


### cohérence station - horaires
- exemple de station avec plusieurs types d'horaire (15)

In [80]:
itinerance_init.loc[itinerance_init.id_station_itinerance == 'FRS23D2302001', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified


### cohérence station - deux-roues
- exemple de station avec plusieurs indications d'accessibilité deux-roues (583)

In [81]:
itinerance_init.loc[itinerance_init.id_station_itinerance == 'FRV75PPX1716', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
105527,108906,supervision-ev.france@totalenergies.com,Belib',"[2.3242009, 48.8957574]",47 Rue Navier 75017 Paris,FRV75PPX1716,Paris | Rue Navier 47,Voirie,7,Accès libre,24/7,TRUE,FRV75EPX17161,2023-07-06,2024-10-17T12:07:34.504000+00:00
105528,108907,supervision-ev.france@totalenergies.com,Belib',"[2.3242009, 48.8957574]",47 Rue Navier 75017 Paris,FRV75PPX1716,Paris | Rue Navier 47,Voirie,7,Accès libre,24/7,TRUE,FRV75EPX17162,2023-07-06,2024-10-17T12:07:34.504000+00:00
105529,108908,supervision-ev.france@totalenergies.com,Belib',"[2.3242009, 48.8957574]",47 Rue Navier 75017 Paris,FRV75PPX1716,Paris | Rue Navier 47,Voirie,7,Accès libre,24/7,FALSE,FRV75EPX17163,2023-07-06,2024-10-17T12:07:34.504000+00:00
105530,108909,supervision-ev.france@totalenergies.com,Belib',"[2.3242009, 48.8957574]",47 Rue Navier 75017 Paris,FRV75PPX1716,Paris | Rue Navier 47,Voirie,7,Accès libre,24/7,FALSE,FRV75EPX17164,2023-07-06,2024-10-17T12:07:34.504000+00:00
105531,108910,supervision-ev.france@totalenergies.com,Belib',"[2.3242009, 48.8957574]",47 Rue Navier 75017 Paris,FRV75PPX1716,Paris | Rue Navier 47,Voirie,7,Accès libre,24/7,FALSE,FRV75EPX17165,2023-07-06,2024-10-17T12:07:34.504000+00:00
105532,108911,supervision-ev.france@totalenergies.com,Belib',"[2.3242009, 48.8957574]",47 Rue Navier 75017 Paris,FRV75PPX1716,Paris | Rue Navier 47,Voirie,7,Accès libre,24/7,FALSE,FRV75EPX17166,2023-07-06,2024-10-17T12:07:34.504000+00:00
105533,108912,supervision-ev.france@totalenergies.com,Belib',"[2.3242009, 48.8957574]",47 Rue Navier 75017 Paris,FRV75PPX1716,Paris | Rue Navier 47,Voirie,7,Accès libre,24/7,FALSE,FRV75EPX17167,2023-07-06,2024-10-17T12:07:34.504000+00:00
